# <center>SMP 2023 ChatGLM金融大模型挑战赛高分方案分享

## <center>Part 1.竞赛方案介绍与文档文档匹配过程

## 一、赛事概览与复盘策略

### 1.SMP 2023 ChatGLM金融大模型挑战赛赛事背景

&emsp;&emsp;SMP 2023 ChatGLM金融大模型挑战赛在第十一届全国社会媒体处理大会（SMP2023）上于2023年在合肥成功举办，由中国中文信息学会社会媒体处理专委会主办，智谱 AI、安硕信息、阿里云和魔搭社区联合承办，天池平台为指定赛事平台，**是我国举办的第一场开源大模型竞赛**。

![image-20230911092111893](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751938.png)

&emsp;&emsp;**本次竞赛明确要求选手仅使用ChatGLM2-6B模型参赛，并为获胜者提供了包括奖金、模型/API使用权以及高效算力在内的多重奖励。**作为国内首场大型大模型竞赛，同时也是以国内最具潜力的ChatGLM模型竞赛，而赛题本身又是时下最热门的大模型技术应用方向——本地知识库问答，因此，自开赛以来便吸引了一众大模型技术爱好者参加。截止复赛开始阶段，总共有将近2300只队伍参赛，盛况空前。赛题详情：[天池官网：SMP 2023 ChatGLM金融大模型挑战赛](https://tianchi.aliyun.com/competition/entrance/532126/introduction?spm=a2c22.12281957.J_3941670930.4.605a3b74eRzY3c&from=search)

&emsp;&emsp;为了印证技术实力、同时了为了更好的为学员提供优质学习内容，我们团队也第一时间参与，并组织学员组成战队参与竞赛，并最终有两只队伍成功杀入复赛。本次案例课程的内容就是基于这两只复赛战队在初赛阶段的方案基础之上进行的整理而成，旨在快速帮助课程学员树立大模型技术的企业级应用全流程。

> 需要注意的是，截止目前，竞赛官方只在初赛中间阶段邀请部分高分战队进行了技术方案分享，此外并没有任何公开任何方案。因此本次案例课程所涉及内容目前为止仍然属于内部分享。

### 2.基于公司财报的金融问答机器人

&emsp;&emsp;围绕本地知识库问答，是目前大模型技术应用备受关注的领域。正如此前课程介绍的那样，受到大模型训练的技术门槛和资金成本限制，大多数公司都无力从头开始预训练大模型，而在使用通用大模型时，通用大模型的原生知识库又存在专业性和实效性不足的问题。因此，如何通过技术手段，使得大模型能够围绕某个特定领域的知识库进行问答，就成了非常热门的大模型技术应用方向。而本次竞赛的要求，本质上就是围绕金融领域文本数据（各上市公司财报），构建一个基于ChatGlm2-6B模型的金融问答机器人。

&emsp;&emsp;而如何构建一个特定知识领域的问答机器人，此前的课程也略有介绍。总的来说技术路线分为两类，其一是借助Embedding+提示工程方法，在每次提问前实时匹配与之相关的文档作为背景信息提前输入，然后再进行回答。当然，这个过程也会涉及到非常多的文本划分和处理的方法，整体流程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202309202022438.png" alt="e31d438adbdf97ad45fa243e86335d5" style="zoom:33%;" />

&emsp;&emsp;具体实现流程如下：首先，需要构建一个知识库。这个知识库可以来源于多种文件格式，如PDF、Word、TXT，甚至是视频中提取出的文本（①）。无论知识库的来源如何，关键是将其内容转化为连贯的文本，例如一个包含一万个自然段的长字符串。（②）

&emsp;&emsp;接下来，需要将这些文本分割成不同的段落或区块（③）。每个区块可以是一个自然段，或根据字数进行划分。这样，将得到一万个独立的文本区块。（假设有一万个段落）

&emsp;&emsp;随后，将每个文本区块通过Embedding算法转化为一个高维向量（④）。如果有一万个这样的区块，那么将得到一万个高维向量。例如，定义每个向量是512维，那么最终会得到一个一万行、512列的向量矩阵，这就是向量库。（⑤）

&emsp;&emsp;上述准备完毕后，当用户提出一个问题时，首先会将该问题通过相同的文本嵌入算法转化为一个512维的向量（⑥）。随后，将此向量与向量库中的一万个向量进行相似度比对，从而识别出与之最为接近的k个向量。这k个相似的向量对应着原知识库中的k个段落。（⑦）

&emsp;&emsp;紧接着，将这k段内容提取出来作为上下文信息，结合这些上下文信息和用户的问题，构建一个完整的提示输入（⑧）到ChatGLM模型中（⑨）。最后，模型会基于这个提示输出相应的答案。（⑩）

&emsp;&emsp;理解了这个过程后，我们可以将其对应到本文的第二个复盘目标：问题与文档标题的匹配。该操作过程与红框内描述的步骤相符，如下：

&emsp;&emsp;而另外一种方法则是进行模型微调。从技术效果上限来说，由于微调会永久修改模型参数，因此能够让模型“永久”记住某些信息，但于此同时微调的技术门槛和算力消耗也远比提示工程复杂很多。因此，大多数本地知识库问答机器人构建过程中，都会优先考虑Embedding+提示工程方法，然后再考虑进行模型微调。

&emsp;&emsp;当然，除了这些看起来更加硬核的技术应用外，在构建本地知识库问答机器人的过程中，还有一项至关重要的工作、甚至是会耗费和占用一半以上实际工作时间的事项，那就是数据清洗。无论是提示工程还是模型微调，数据清洗都是必不可少的环节。而若是进行提示工程，则需要按照一定格式对文档进行整理和内容划分，而如果是进行微调，则还需进一步整理高质量的问答数据。

&emsp;&emsp;因此，当前案例在进行分享的过程中，也是按照数据清洗、Embedding+提示工程、模型微调的整体逻辑来进行的内容编排。而在每个大环节下，还会有非常多具体的技术落地细节介绍。

### 3.赛题数据评分标准介绍

&emsp;&emsp;此次竞赛的核心任务是要求参赛者**基于ChatGLM2-6B模型构建金融问答系统，精准回答用户的金融问题**。同时，竞赛规定禁止使用其他大语言模型，但允许选手利用公开的外部数据微调模型，并结合向量数据库等先进技术。正如此前介绍，赛题数据就是近几年部分公司财报，总共包含11588份财报文档，而每一份财报内容都有几十页甚至上百页内容：

![Snipaste_2023-09-01_10-38-22](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751948.jpg)

**而关于评估标准，它根据模型的能力和复杂度划分为初级、中级和高级三个层次，每个层次的题目都有其特定的分值**，如下：

- **初级：数据基本查询（40分）**

&emsp;&emsp;利用提供的ChatGLM2-6B开源模型和上市公司年报原始数据，并以此为基础创建信息问答系统，使系统具备解决基本查询的能力。

![image-20230911102238482](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751939.png)

- **中级：数据统计分析查询（30分）**

&emsp;&emsp;在已具备基础查询能力的前提下，进一步优化系统，使其能够根据各种指标进行统计分析，并返回计算后的答案。

![image-20230911102708195](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751940.png)

- **高级：开放性问题（30分）**

&emsp;&emsp;除了满足基础与中级的数据查询需求，系统需要进一步具备解答开放性问题的能力。

> 例：能够根据具体的查询，判断某公司在2021年的主要研发项目是否涉及到国家创新领域，如新能源技术、人工智能等，并为用户提供详尽的背景信息和证据。


<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202309202023082.jpg" alt="4bc6aaa361ec81876722c3d63a1f9e4" style="zoom:33%;" />

&emsp;&emsp;更多赛题评分说明，参看[SMP 2023 ChatGLM金融大模型挑战赛**评测任务概述**](https://tianchi.aliyun.com/competition/entrance/532126/information)

### 4.赛题难点分析

&emsp;&emsp;在金融领域，构建一个能够像专家一样解读上市公司年报对话交互的智能系统一直是人工智能发展的重要目标，除去对ChatGLM-6B模型数值计算能力的担忧，转观金融垂直领域，尽管当前的AI系统在日常文本对话中已展现出卓越的性能，但面对复杂且细粒度的金融场景交互，其效能仍有提升空间。年报作为金融领域的核心文件，全面展现了公司的经营状况、财务健康以及未来展望。其内容涵盖了公司组织、员工构成、业务发展、财务数据和策略规划等众多方面。此外，年报还深入分析了外部市场态势、行业趋势和公司的竞争地位，为投资者和其他相关方提供了宝贵的决策依据。

&emsp;&emsp;**然而，解读年报并非易事**。年报通常包含大量的专业金融和商业术语，甚至一些暗含的信息，如潜在的风险或利好，所以要做好这项任务可以想象到：
- **难点1：如何理解和解析年报中的专业金融和商业术语。**
- **难点2：如何捕捉到年报中的暗含信息，例如潜在的风险或机会。**
- **难点3：如何模仿专家的深度理解，让AI模型不仅提取信息，还能深度解析和评估。**
- **难点4：如何微调数据集并构建适合金融领域的向量库。**
- **难点5：如何有效地训练和微调大模型，确保其在金融领域的应用效果。**
- .............

&emsp;&emsp;所以在现有大模型的基础上进行性能增强绝非易事。我们需要采用微调大模型、实现大小模型的协同工作、并结合向量数据库等前沿技术，以赋予AI模型解读年报和深度金融分析的专家级能力。这样的挑战，无疑是一项巨大的任务。

### 5.其他准备

#### 5.1 硬件环境

&emsp;&emsp;以下是本次竞赛过程中老师赛队运行环境：
- **CPU**：Intel 6133 2.5GHz，20核40线程，双路
- **内存**：192GB，2666MHz
- **GPU**：NVIDIA RTX 3090 24G，共4张
- **硬盘**：至少保留150GB的可用空间

&emsp;&emsp;若要完整复现本次案例分享中的全部代码，则推荐以下最低配置：

不加载模型(CPU/GPU均不加载)        
CPU版：可用内存 4GB(除系统、其他程序)

text2vec-large-chinese        
GPU版：可用内存 4GB(除系统、其他程序)+GPU 4GB        
CPU版：可用内存 8GB(除系统、其他程序)

ChatGLM2-6B (FP16)        
GPU版：可用内存 6GB(除系统、其他程序)+GPU 14GB        
CPU版：可用内存 4GB(除系统、其他程序) + model 需要大概 32GB 内存

#### 5.2 开源模型与其他组件安装

&emsp;&emsp;此外需要注意的是，由于本次案例分享时间有限，暂时不会在课程进行时介绍案例所涉及到的ChatGLM等开源大模型和相关组件的安装过程。这部分安装工作我们会在对应使用到这部分组件时提供详细的安装文档，以供大家手动复时使用。

#### 5.3 本节内容概览

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202309202025071.png" alt="b3bfefdfea66cdc8409a9603a921e7d" style="zoom:33%;" />

## 二、数据探索与结构解析

### 1.数据说明

&emsp;&emsp;竞赛官方已在ModelScope社区发布了2019年至2021年间部分上市公司的年度报告数据集，**这一数据集涵盖了11588份详实的PDF年报，需要依据这批PDF资料来搭建所需的数据库或向量库**，官网中对数据情况做了详细的说明：[ChatGLM评估挑战赛-金融赛道数据集](https://modelscope.cn/datasets/modelscope/chatglm_llm_fintech_raw_dataset/summary)

![image-20230911113626979](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751942.png)

&emsp;&emsp;其中需要关注以下几点：

1. **数据大小**

&emsp;&emsp;在官网的数据集介绍模块给出了年报数据的相关描述及推荐的处理过程，**需要注意：全部的PDF 文件约69.3GB，如下载请确保有足够的硬盘空间**

![image-20230911113946834](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751943.png)

2. **数据预览**

&emsp;&emsp;鉴于69GB的数据量较大，可能导致下载不便，官方在首页已经提供了一个预览下载链接。这允许用户单独下载特定的PDF，从而方便地预览数据内容。

![image-20230911121922471](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751944.png)

&emsp;&emsp;**需要注意：此链接仅提供前100个文件的预览。如需下载完整的PDF数据文件，请按照官方指引进行。**

![image-20230911122142269](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751945.png)

3. **加载方式**

&emsp;&emsp;官方提供了三种获取年报PDF数据集的方式，如下：

![image-20230911114401998](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751946.png)

这里推荐采用加载方式1进行加载，加载方式2可能出现Hash值不匹配的过程。

### 2.数据下载

- **Step 1: 将整个数据集下载到本地**

&emsp;&emsp;为了实现此目标，我们采用了git作为首选的数据加载方法(加载方式一)。但是需要注意：若选择此方式，请确保您的本地磁盘有超过70GB的可用空间。

&emsp;&emsp;以下是下载好的数据集情况：

![Snipaste_2023-09-01_10-38-22](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751948.jpg)

> 如出现数据集下载问题，可查看：https://tianchi.aliyun.com/forum/post/571986获取帮助

这里需要注意的是，文档数据包括两个类型，不同类型文档后续需要采用不同的处理办法：
- python-pdf可读取(word转pdf)
- python-pdf不可直接读取(扫描版、需要OCR识别)

- **Step 2：查看数据情况**

&emsp;&emsp;首先一次性导入必要的第三方库，如下：

In [4]:
# 导入所需库
import os
import gc
import sys
import time
import json
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from pprint import pprint

&emsp;&emsp;创建一个函数，该函数遍历指定目录，返回与给定格式（默认为`.pdf`）匹配的文件名列表，并显示搜索进度。

In [5]:
def search_file(path: str,
                format: str = '.pdf'):
    """
    Find files within a folder limited by defind format
    return ---> tuple = (path, filename)
    """
    path_files = []
    # for (ipath, ifloder, ifile) in os.walk(path):
    for entry in tqdm(os.scandir(path), ncols=100):
        path_files.append(entry.path.split('/')[-1]) if format in entry.path.split('/')[-1] else ''
    print('Total file ---> %d' % len(path_files))
    return path_files   

&emsp;&emsp;执行函数，打印查看前三个金融文档(.pdf)文件的文档名称：

In [6]:
# 此处修改为你本地的PDF下载路径
allpdf_path = r"./smp_data/"
# 读取所有 PDF 
pdf_name_list = search_file(path = allpdf_path, format='.pdf')
pdf_name_list[:3]

5it [00:00, ?it/s]

Total file ---> 5


['2020-01-21__江苏安靠智能输电工程科技股份有限公司__300617__安靠智电__2019年__年度报告.pdf',
 '2020-01-23__浙江海翔药业股份有限公司__002099__海翔药业__2019年__年度报告.pdf',
 '2020-02-08__焦点科技股份有限公司__002315__焦点科技__2019年__年度报告.pdf']

&emsp;&emsp;从给定的金融文档(.pdf)文件名中，可以观察到一个相对统一和规范的命名格式，具体为：**使用“__”来分割，包含以下六个部分：**
- 0: 财报发布如期，财报内容的下一年初；
- 1: 上市公司名称（全称）；
- 2: 上市公司股票代码（6 位数字）；
- 3: 上市公司简称（如：医药、食品等）；
- 4: 财报年份（四位数字+'年'）；
- 5: 报告类别，所有PDF都是年度报告；

&emsp;&emsp;这种规范的文件命名方式极大地方便了后续的处理，免去了对文件名称的额外处理步骤。当本地的train数据准备就绪后，我们就可以开始前期的数据清洗工作。

### 3.文档解析

&emsp;&emsp;当所有的金融文档(.pdf)文件成功下载至本地后，首先需要考虑的是：**对于规范且具有固定模式的文件命名，将其拆分并转化为结构化数据是至关重要，且必须要做的步骤**。

&emsp;&emsp;例如，对于文件名 `2020-01-21__江苏安靠智能输电工程科技股份有限公司__300617__安靠智电__2019年__年度报告.pdf`，我们可以将其拆分为以下几个部分：

- 财报发布日期：2020-01-21
- 上市公司全称：江苏安靠智能输电工程科技股份有限公司
- 上市公司股票代码：300617
- 上市公司简称：安靠智电
- 财报年份：2019年
- 报告类别：年度报告

&emsp;&emsp;这样的操作不仅增强了数据的清晰性和可读性，还为后续的数据处理和分析提供了更加方便的基础，比如在后续的分析或模型训练中，你可能只对某些特定的字段感兴趣，如“财报年份”或“上市公司名称”，拆分存储使得这种选择性的操作变得更加容易。具体操作如下：

- **Step 1：转换成pd.DataFrame 格式**

&emsp;&emsp;将规范的文件命名拆分并存储为 `DataFrame` 格式可以实现数据的结构化管理，便于后续的查询、分析和可视化操作。代码如下：

In [7]:
# 将数据转换为 pd.DataFrame 格式
df_pdf_name = pd.DataFrame(pdf_name_list, columns=['pdf_name'])
df_pdf_name.index.rename('pdf_idx', inplace=True)   # 后续会涉及到很多表，对每张表采取不同的index_label
df_pdf_name.head()

,pdf_name
pdf_idx,
0,2020-01-21__江苏安靠智能输电工程科技股份有限公司__300617__安靠智电__...
1,2020-01-23__浙江海翔药业股份有限公司__002099__海翔药业__2019年_...
2,2020-02-08__焦点科技股份有限公司__002315__焦点科技__2019年__年...
3,2020-02-14__平安银行股份有限公司__000001__平安银行__2019年__年...
4,2020-02-20__上海家化联合股份有限公司__600315__上海家化__2019年_...


- **Step 2：将pd.DataFrame格式的文件名进行拆分**

&emsp;&emsp;对于有规律的文件名或字符串，如果希望将其分解成多个部分并分别存储，**Pandas的 `.loc` 方法提供了一种方式来实现这种需求，它允许根据行和列的标签对DataFrame进行索引，并对多个元素进行同时赋值**

&emsp;&emsp;比如：创建一个简单的 DataFrame `df_temp`，包含单列数据 'A', 'B', 'C', 和 'D'。

In [8]:
# 创建一个测试用例 df_temp
df_temp = pd.DataFrame(list('ABCD'))
df_temp

,0
0,A
1,B
2,C
3,D


&emsp;&emsp;通过使用 `.loc` 方法对 DataFrame 进行多项赋值，即：将第0行和第1行的值更改为 'a' 和 'b'。

In [9]:
df_temp.loc[[0,1],0] = "a", "b"

In [10]:
df_temp

,0
0,a
1,b
2,C
3,D


&emsp;&emsp;同时`pandas` 提供了非常方便的字符串处理功能。若想根据某个字符对字符串进行拆分并将其拆分到不同的列中，可以使用 `str.split` 方法，所以我们这样定义拆分函数：

In [11]:
def Split_PDF_name(x:str, split_:str="__", format:str=".pdf"):
    x=x.split(format)[0]
    # print(x, len(x.split(split_)), x.split(split_))
    return x.split(split_)

# df_pdf_name['pdf_name'].apply(Split_PDF_name)
col_name_list = ['pub_time', 'all_name', 'code', 'short_name', 'year', 'type']
for idx, item in tqdm(df_pdf_name.iterrows()):
    df_pdf_name.loc[idx,col_name_list] = Split_PDF_name(item['pdf_name'])
df_pdf_name.head()

5it [00:00, 836.15it/s]


,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
0,2020-01-21__江苏安靠智能输电工程科技股份有限公司__300617__安靠智电__...,2020-01-21,江苏安靠智能输电工程科技股份有限公司,300617,安靠智电,2019年,年度报告
1,2020-01-23__浙江海翔药业股份有限公司__002099__海翔药业__2019年_...,2020-01-23,浙江海翔药业股份有限公司,002099,海翔药业,2019年,年度报告
2,2020-02-08__焦点科技股份有限公司__002315__焦点科技__2019年__年...,2020-02-08,焦点科技股份有限公司,002315,焦点科技,2019年,年度报告
3,2020-02-14__平安银行股份有限公司__000001__平安银行__2019年__年...,2020-02-14,平安银行股份有限公司,000001,平安银行,2019年,年度报告
4,2020-02-20__上海家化联合股份有限公司__600315__上海家化__2019年_...,2020-02-20,上海家化联合股份有限公司,600315,上海家化,2019年,年度报告


&emsp;&emsp;应用拆分函数并使用.loc 方法对 DataFrame 完成多项赋值，代码如下：

In [12]:
# df_pdf_name['pdf_name'].apply(Split_PDF_name)
col_name_list = ['pub_time', 'all_name', 'code', 'short_name', 'year', 'type']
for idx, item in tqdm(df_pdf_name.iterrows()):
    df_pdf_name.loc[idx,col_name_list] = Split_PDF_name(item['pdf_name'])
df_pdf_name.head()

5it [00:00, 1251.81it/s]


,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
0,2020-01-21__江苏安靠智能输电工程科技股份有限公司__300617__安靠智电__...,2020-01-21,江苏安靠智能输电工程科技股份有限公司,300617,安靠智电,2019年,年度报告
1,2020-01-23__浙江海翔药业股份有限公司__002099__海翔药业__2019年_...,2020-01-23,浙江海翔药业股份有限公司,002099,海翔药业,2019年,年度报告
2,2020-02-08__焦点科技股份有限公司__002315__焦点科技__2019年__年...,2020-02-08,焦点科技股份有限公司,002315,焦点科技,2019年,年度报告
3,2020-02-14__平安银行股份有限公司__000001__平安银行__2019年__年...,2020-02-14,平安银行股份有限公司,000001,平安银行,2019年,年度报告
4,2020-02-20__上海家化联合股份有限公司__600315__上海家化__2019年_...,2020-02-20,上海家化联合股份有限公司,600315,上海家化,2019年,年度报告


- **Step 3：数据持久化**

&emsp;&emsp;经过拆分赋值处理后，可以利用 Pandas 的 `to_csv` 方法将数据保存到本地，为后续操作提供便利。

> 提示：在开发测试中，建议为每个文件名添加时间戳，以避免不同版本的数据被意外覆盖。

In [15]:
# fn = "output/0.pdf_name-%s.csv"%stamp()   # 对每份文件加入时间戳，防止覆盖
# df_pdf_name.to_csv(fn, sep=";", index_label="pdf_idx")
# print(fn)
# output/0.pdf_name-20230901-120201.csv

&emsp;&emsp;上述步骤就完成了 PDF 文件名的解析和拆分，并将处理后的数据保存到了本地。**至此，暂时完成了对金融文档(.pdf)的初步清洗处理。**

## 三、问题解析与分类

### 1.问题解析

&emsp;&emsp;理解了上述流程后，我们可以明确首要任务是解析问题。在这次竞赛中，官方提供的元数据中已经包括了`test_questions.json`，具体内容如下：

![image-20230911154004629](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751954.png)

&emsp;&emsp;点击查看可看到具体的问题：

![image-20230911154817671](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309111751955.png)

在这份`test.json`测试集中，一共包含了5000条字典形式的数据。

> 注意：如果想在竞赛成功提交结果，需要确保文件满足以下条件：
> - 文件编码：UTF-8
> - 换行符：UNIX

>  不满足上述格式条件的提交可能会遭遇“长度不符合”等错误提示。

&emsp;&emsp;数据集中的问题主要分为两类：

1. 大部分问题与类型1~3相似，主要询问某公司在特定年份的具体财务指标。
2. 少数问题属于类型4，即与财务概念或技术问题相关。这类问题并不依赖数据集中的PDF文件内容。

&emsp;&emsp;尝试使用代码解析问题(questions)，具体步骤如下：

**Step 1: 读取test_questions.json文件**

&emsp;&emsp;从`test_questions.json`文件中读取数据，然后将其转换为一个以`q_idx`为索引的Pandas DataFrame，代码如下：

In [20]:
testjson_path = 'data/test_questions.json'
tests = []
with open(testjson_path, 'r+', encoding="utf-8") as file:
    for i in file.readlines():     # 对每一行进行读取
        tests.append(json.loads(i)) # 采用 json.loads 加载
df_tests = pd.DataFrame(tests).set_index('id') # 重置 id_label
df_tests.index.rename("q_idx", inplace=True)

In [22]:
df_tests.head(10)

,question
q_idx,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...
2,2019年四方科技电子信箱是什么?
3,研发费用对公司的技术创新和竞争优势有何影响？
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？
5,苏州科达在2020年的博士及以上人员数量是多少?
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。


- **Step 2：问题数据分类探索**

&emsp;&emsp;通过观察这些数据，可以注意到年份在所有问题中是一个常见且显著的元素。因此，可以尝试先检查每个问题中是否包含年份，让其按照年份先将问题初步分类，代码如下：

In [27]:
def include_year(x, str_list=list([2019, 2020, 2021])):
    list_ = [i for i in str_list if str(i) in str(x)]
    return list_, len(list_)

year_lists = []
year_counts = []

for _, item in df_tests.iterrows():
    year_list, year_count = include_year(item['question'])
    year_lists.append(year_list)
    year_counts.append(year_count)

df_tests['year_list'] = year_lists
df_tests['year_count'] = year_counts
df_tests.head()

,question,year_list,year_count
q_idx,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,[2020],1
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,[2019],1
2,2019年四方科技电子信箱是什么?,[2019],1
3,研发费用对公司的技术创新和竞争优势有何影响？,[],0
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,[2020],1


&emsp;&emsp;统计根据年份分类后，各问题类型的数量占比：

In [28]:
# 查看 各分类数量
df_tests["year_count"].value_counts()

1    4633
0     300
2      67
Name: year_count, dtype: int64

&emsp;&emsp;根据返回结果来看，可以将问题大致分类为三种情况：

- **包含年份个数为0的问题，代表问题中未提到任何年份，即得到空列表**
- **包含年份个数为1的问题，问题中只提到了一个年份，如[2019]**
- **包含年份个数为2的问题，问题中涉及到两个年份，如[2019，2020]**

&emsp;&emsp;进一步的具体分析每一类问题的特征。首先对于包含年份个数为0的问题，未提到年份的问题有300个。

In [30]:
df_tests[df_tests['year_count'] == 0]

,question,year_list,year_count
q_idx,,,
3,研发费用对公司的技术创新和竞争优势有何影响？,[],0
19,合并财务报表的编制方法有哪些？,[],0
54,什么是存货周转率？,[],0
106,重要会计政策和会计估计的变更是什么？,[],0
114,什么是营运资本？,[],0
...,...,...,...
4956,什么是短期借款？,[],0
4958,生物资产是指什么？,[],0
4978,什么是其他应收款？,[],0


&emsp;&emsp;这类问题主要是询问一些财务和会计概念或者是对某些特定术语的解释。例如：“研发费用对公司的技术创新和竞争优势有何影响？”或“什么是存货周转率？”，不包含具体的年份信息，因为它们大多数是关于概念性知识的询问，而不是某一特定时间段的财务数据。

&emsp;&emsp;**简而言之，这类问题更多地是对某些财务或会计术语、概念、政策或方法的通用性提问，不依赖于特定的年报数据或时间段。**

&emsp;&emsp;其次，对于类比1来说，涉及1个年份的问题有4633个，占比最大。

In [31]:
df_tests[df_tests['year_count'] == 1]

,question,year_list,year_count
q_idx,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,[2020],1
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,[2019],1
2,2019年四方科技电子信箱是什么?,[2019],1
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,[2020],1
5,苏州科达在2020年的博士及以上人员数量是多少?,[2020],1
...,...,...,...
4994,2021年鹏欣环球资源股份有限公司总资产增长率为多少?保留2位小数。,[2021],1
4995,苏州东微半导体股份有限公司在2021年归属于母公司所有者的净利润是多少元?,[2021],1
4996,2020年顺威股份研发费用增长率是多少?保留2位小数。,[2020],1


&emsp;&emsp;对于只涉及一个年份（占比为1）的问题，这类问题通常非常具体，指向某一家公司在某一年度的具体内容或数据，且大多数问题都与某一特定的财务指标或数据点有关，例如“资产负债比率”、“博士及以上人员数量”或“研发费用”等，另外这类问题比较需要关注的一点是：**问题通常会明确指出要查询的公司名称，如“金宇生物技术股份有限公司”或“江化微”。**

&emsp;&emsp;简而言之，这类问题的特点是明确、具体，并针对某一家公司的某一年的特定内容或数据进行查询。这为后续的数据检索、筛选和分析提供了明确的方向和依据。

&emsp;&emsp;最后来看包含年份个数为2的问题，涉及2个年份的问题有67个，占比最小。

In [32]:
df_tests[df_tests['year_count'] == 2]

,question,year_list,year_count
q_idx,,,
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"[2019, 2020]",2
214,请对比2020年和2021年，永清环保股份有限公司的法定代表人是否相同?,"[2020, 2021]",2
289,对比2020年，2021年中航电子的法定代表人是否相同?,"[2020, 2021]",2
368,在2020年，上海开开实业股份有限公司的法定代表人与2019年相比相同吗?,"[2019, 2020]",2
490,山东阳谷华泰化工股份有限公司在2019-2021年间，法定代表人是否都相同?,"[2019, 2021]",2
...,...,...,...
4617,中金岭南2021年的法定代表人与2020年相比是否相同？,"[2020, 2021]",2
4672,上海华鑫股份有限公司2020年法定代表人对比2019年是否相同?,"[2019, 2020]",2
4738,对比2020年，2021年江苏蔚蓝锂芯股份有限公司的法定代表人是否相同?,"[2020, 2021]",2


&emsp;&emsp;对于涉及两个年份的问题，这类问题主要是进行两个年份之间的对比，例如比较公司的法定代表人在两个不同年份是否相同。与单年份问题类似，这类问题也会明确指出要查询的公司名称。**这类问题需要特别注意：大多数问题都是针对某个特定的数据点进行查询，如“法定代表人”。**

&emsp;&emsp;简而言之，这类问题的核心在于对两个年份的某一特定数据或内容进行比较。这种对比性的查询可能需要从两个不同的数据点中检索信息，并进行相应的比较。

**Step 3：问题分类**

&emsp;&emsp;在对问题(questions)特点进行分析后，可以清晰地看到，**包含年份个数为2的问题显然是涉及两个年份的内容而产生的,这意味着为了准确回答这些问题，需要同时考虑两个年份的PDF内容**。这是后续处理中一个重要的细节。

&emsp;&emsp;因此，我们选择根据问题中涉及的年份数量对问题进行分类。具体地说，不涉及具体年份的问题、也就是财务概念相关的问题被标记为“notion”，而需要查询年报内容的问题则被标记为“pdf”。以下是实现这一划分的代码：

In [33]:
df_tests['q_type'] = df_tests['year_count'].apply(lambda x: 'notion' if x == 0 else 'pdf')
df_tests.head()

,question,year_list,year_count,q_type
q_idx,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,[2020],1,pdf
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,[2019],1,pdf
2,2019年四方科技电子信箱是什么?,[2019],1,pdf
3,研发费用对公司的技术创新和竞争优势有何影响？,[],0,notion
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,[2020],1,pdf


- **Step 4：本地存储**

&emsp;&emsp;经过处理后，同样可以将结果保存到本地，以便于后续的使用和调用，代码如下：

In [34]:
# fn = "output/0.Question-%s.csv"%stamp()   # 对每份文件加入时间戳，防止覆盖
# df_tests.to_csv(fn, sep=";", index_label="q_idx")
# print(fn)
# output/0.Question-20230901-151445.csv

# df_tests = pd.read_csv("output/0.Question-20230901-151445.csv", sep=";", index_col="q_idx")
# df_tests.head()

这里需要注意的是，对于notion类问题，是可以直接调用ChatGLM模型进行回答的，无需额外输入参考文件。

## 1.3 notion问题直接询问 ChatGLM2-6B

In [21]:
from typing import *
import torch
def torch_gc():
    # 定义一个名为`torch_gc`的函数，用于清空GPU缓存。这个函数只在GPU可用时执行。
    if torch.cuda.is_available():
        with torch.cuda.device(CUDA_DEVICE):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
        
def auto_configure_device_map(num_gpus: int) -> Dict[str, int]:
    # 定义一个名为`auto_configure_device_map`的函数，接受一个整数参数`num_gpus`，用于指定有多少GPU可用。函数返回一个字典，其中包含每个层及其对应的GPU编号。
    num_trans_layers = 28
    per_gpu_layers = 30 / num_gpus

    device_map = {
        'transformer.embedding.word_embeddings': 0,
        'transformer.encoder.final_layernorm': 0,
        'transformer.output_layer': 0,  # glm2 改动
        'transformer.rotary_pos_emb': 0, # glm2 改动
        'lm_head': 0
    }

    used = 2
    gpu_target = 0
    # 使用一个for循环遍历所有的transformer层（num_trans_layers），并根据per_gpu_layers计算每个GPU应该处理的层数。同时，将gpu_target变量更新为当前的GPU编号。
    for i in range(num_trans_layers):
        if used >= per_gpu_layers:
            gpu_target += 1
            used = 0
        assert gpu_target < num_gpus
        device_map[f'transformer.encoder.layers.{i}'] = gpu_target  # 加上encoder
        used += 1

    return device_map

In [22]:
import torch
from transformers import AutoModel, AutoTokenizer

sti = time.time(); print("Load Model ...", end='\t')



num_gpus = torch.cuda.device_count()
model_name_or_path = "/home/util/ChatGLM2-6B"
# 多块gpu
print(f'{num_gpus} GPU')
from accelerate import dispatch_model

tokenizer = AutoTokenizer.from_pretrained("/home/util/ChatGLM2-6B", trust_remote_code=True)
model = AutoModel.from_pretrained(model_name_or_path,
                                    trust_remote_code=True,
                                    ).half()

# 多块gpu将模型的不同层分配到不同的gpu

device_map = auto_configure_device_map(num_gpus)
# print(device_map)
model = dispatch_model(model, device_map=device_map)

print("Done [ %.2f s]\n\n"%(time.time()-sti)) 


/home/util/anaconda3/envs/lc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load Model ...	4 GPU


Loading checkpoint shards: 100%|██████████| 7/7 [00:17<00:00,  2.53s/it]


Done [ 28.71 s]




启动后显存占用情况如下，<br>
4207+3842+3842+3062 = 14,953 MB ~= 15GB<br>
![](image/Snipaste_2023-09-01_15-26-06.jpg)

In [27]:
def get_chat(tokenizer, model, prompt):
    response, history = model.chat(tokenizer, prompt, history=[])
    return response

df_notion = df_tests[df_tests['q_type']=='notion'].copy()
df_notion['answer'] = ''
for idx, item in tqdm(df_notion.iterrows()):
    print(idx, item['question'])
    # print(get_chat(tokenizer, model, prompt=item['question']))
    df_notion.loc[idx, 'answer'] = get_chat(tokenizer, model, prompt=item['question'])
    print('*'*50,"\n")

# 程序正确，人为终止输出;
# 详细结果如下


这里所有财务类问题的问答结果都保存在本地的df_notions_As.csv文件中，部分问答结果展示如下：

In [16]:
df_notion_ans = pd.read_csv('data/df_notions_As.csv', sep=';', index_col='id')
df_notion_ans = df_notion_ans.loc[:,['question', 'answer']]
df_notion_ans.head()

,question,answer
id,,
3,研发费用对公司的技术创新和竞争优势有何影响？,"研发费用是公司用于新产品、新技术或新业务的投资,通常包括研究、设计、开发、测试和推广等环节所..."
19,合并财务报表的编制方法有哪些？,合并财务报表的编制方法包括以下两种：\n\n1. 直接法（Consolidated Fina...
54,什么是存货周转率？,"存货周转率是一个衡量公司在一定时期内销售了多少存货的指标,反映了公司的存货管理效率和经营能力..."
106,重要会计政策和会计估计的变更是什么？,重要会计政策和会计估计的变更是指企业对重要的会计政策或者会计估计进行变更所引起的财务报表变化...
114,什么是营运资本？,"营运资本是指企业用于日常运营和业务的资金,包括现金、存货、在途资金、应收账款和预付账款等。\..."


In [5]:
df_notion_ans.columns

Index(['question', 'q_year', 'q_names', 'match_names', 'answer'], dtype='object')

## 四、问题与文档匹配

&emsp;&emsp;到目前为止，已经成功地完成了测试问题(questions)的解析，并根据年份对问题进行了分类。这一操作为我们提供了一个更加清晰、结构化的问题集，为下一步建立问题(questions)与金融文档(.pdf)标题的对应关系打下了坚实的基础。

&emsp;&emsp;在确保每一个问题都被准确分类后，现在将着手于一个关键的步骤，即**确定哪些问题(questions)与哪些金融文档(.pdf)标题有关联**。这一步骤是至关重要的，因为它将直接决定我们从知识库中提取的上下文信息的质量和相关性，从而影响到最终生成答案的准确性。

&emsp;&emsp;具体如何创建关联呢？

&emsp;&emsp;首先，回顾一下已处理的数据集。第一份数据集是经过结构化解析的文件名，如下所示：

In [36]:
df_pdf_name = pd.read_csv("output/0.pdf_name-20230901-120201.csv", sep=";", index_col="pdf_idx")
df_pdf_name

,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
0,2021-04-24__数源科技股份有限公司__000909__数源科技__2020年__年...,2021-04-24,数源科技股份有限公司,909,数源科技,2020年,年度报告
1,2021-04-19__合兴汽车电子股份有限公司__605005__合兴股份__2020年_...,2021-04-19,合兴汽车电子股份有限公司,605005,合兴股份,2020年,年度报告
2,2021-04-15__昆吾九鼎投资控股股份有限公司__600053__九鼎投资__2020...,2021-04-15,昆吾九鼎投资控股股份有限公司,600053,九鼎投资,2020年,年度报告
3,2020-04-30__嘉凯城集团股份有限公司__000918__嘉凯城__2019年__年...,2020-04-30,嘉凯城集团股份有限公司,918,嘉凯城,2019年,年度报告
4,2020-05-29__北大医药股份有限公司__000788__北大医药__2019年__年...,2020-05-29,北大医药股份有限公司,788,北大医药,2019年,年度报告
...,...,...,...,...,...,...,...
11583,2020-03-24__宁夏银星能源股份有限公司__000862__银星能源__2019年_...,2020-03-24,宁夏银星能源股份有限公司,862,银星能源,2019年,年度报告
11584,2022-04-19__狮头科技发展股份有限公司__600539__狮头股份__2021年_...,2022-04-19,狮头科技发展股份有限公司,600539,狮头股份,2021年,年度报告
11585,2021-03-31__广东金明精机股份有限公司__300281__金明精机__2020年_...,2021-03-31,广东金明精机股份有限公司,300281,金明精机,2020年,年度报告


&emsp;&emsp;接着，还有根据年份分类后的问题类型数据集，如下所示：

In [38]:
df_tests = pd.read_csv("output/0.Question-20230901-151445.csv", sep=";", index_col="q_idx")
df_tests

,question,year_list,year_count,q_type
q_idx,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,[2020],1,pdf
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,[2019],1,pdf
2,2019年四方科技电子信箱是什么?,[2019],1,pdf
3,研发费用对公司的技术创新和竞争优势有何影响？,[],0,notion
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,[2020],1,pdf
...,...,...,...,...
4995,苏州东微半导体股份有限公司在2021年归属于母公司所有者的净利润是多少元?,[2021],1,pdf
4996,2020年顺威股份研发费用增长率是多少?保留2位小数。,[2020],1,pdf
4997,预计负债是指什么？,[],0,notion


&emsp;&emsp;考虑到两份数据集的特性，我们可以从 `df_tests` 的 `question` 字段和 `df_pdf_name`的`pdf_name` 字段中提取的信息中找到一个结合点：即**共同的年份和公司名称可以作为关联的关键。**

&emsp;&emsp;确定了这种策略后，我们接下来的步骤是**对 `question` 和 `pdf_name` 进行Embedding处理，并通过计算余弦相似度来建立它们之间的最佳对应关系。**

> 文本嵌入（Embedding）可以被简单地理解为将信息通过特定算法转化为高维向量，从而使得计算机能够进行快速且有效的计算。

&emsp;&emsp;我们使用 `text2vec-large` 模型来进行文本向量化，将文本内容（无论是单词、句子还是段落）转化为具体的向量表示。

1. text2vec-large介绍 <br>
text2vec-large是一个在大规模语料上预训练得到的通用语义文本嵌入模型,由Facebook AI Research团队训练获得。
主要特点是:
- 训练语料规模大,使用了包含1TB英文维基百科文档的语料。
- 使用CBOW算法进行无监督预训练。
- 可支持多种语言使用。<br><br>
2. 模型结构<br>
text2vec-large主要由输入层、投影层和输出层组成。
- 输入层将词编码为one-hot表示。
- 投影层将高维的one-hot降为低维稠密词向量。
- 输出层进行目标词预测,通过上下文预测目标词实现CBOW训练。<br><br>
3. 文本相似度计算<br>
text2vec-large的词向量可用来计算两个文本之间的相似度。主要步骤是:
- 对两个文本分词,获得词列表。
- 查询词向量获得每个词的向量。
- 对两个文本分别求词向量平均,得到文本向量。
- 计算两个文本向量之间的余弦相似度。
- 相似度越高表示两个文本在语义上越相似。<br><br>
text2vec-large训练良好的词向量,可以更加准确地判断语义相似度,可广泛用于相似句匹配、语义搜索、文本聚类等任务中。是自然语言处理中一个非常强大的基础模型

以下是具体的操作步骤：

- **Step 1：导入必要库**

In [41]:
import gradio as gr
import nltk
from langchain.embeddings import OpenAIEmbeddings  
import sentence_transformers
from duckduckgo_search import ddg
from duckduckgo_search.utils import SESSION
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader  
from langchain.embeddings.huggingface import HuggingFaceEmbeddings 
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores import FAISS

import faiss
from fastchat.conversation import (compute_skip_echo_len,
                                   get_default_conv_template)
from fastchat.serve.inference import load_model as load_fastchat_model
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, AutoConfig

- **Step 2：启动text2vec-large 模型**

&emsp;&emsp;加载预训练的`text2vec-large-chinese`模型并初始化一个向量化工具，代码如下：

In [ ]:
emb_path = "/home/util/text2vec-large-chinese"
embeddings = HuggingFaceEmbeddings(model_name=emb_path, )
embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name, device='cuda:0')

No sentence-transformers model found with name ./text2vec-large-chinese/. Creating a new one with MEAN pooling.
No sentence-transformers model found with name ./text2vec-large-chinese/. Creating a new one with MEAN pooling.


In [ ]:
def get_emb(txt:str=''):
    emb = embeddings.client.encode(txt)
    faiss.normalize_L2(emb[np.newaxis,:]) # 正则化
    return emb

- **Step 3：调用测试**

&emsp;&emsp;为了验证创建的嵌入模型是否能有效地捕获文本之间的语义关系，我们进行一个简单的测试：比较两组词汇对的相似度，代码如下

In [41]:
## 一个小测试

print(np.dot(get_emb('苹果'), get_emb('熊猫')))
# 0.2303976
# 看来苹果和熊猫的亲密关系比较疏远

print(np.dot(get_emb('苹果'), get_emb('apple')))
# 0.8299886
# 非常相近

AssertionError: Torch not compiled with CUDA enabled

&emsp;&emsp;从结果上看，是没有任何问题的，所以我们可以将其应用到问题(questions)与金融文档(.pdf)标题的嵌入表示上。

&emsp;&emsp;具体来说，我们计划从`df_tests` 的 `question` 字段中选取5个问题(questions)，并为这5个问题在`df_pdf_name`中找到对应的`pdf_name`值，然后将这5个问题和其对应的`pdf_name`将分别进行Embedding处理，最后利用这些嵌入向量进行相似度匹配测试。

&emsp;&emsp;选择5个问题进行验证的原因是为了确保我们的方法在实际应用场景中是可行的。通过对一小部分数据进行测试，可以更快速地发现潜在问题，并在必要时进行调整。此外，这种方法还可以帮助我们预估整个数据集处理的时间和资源消耗。只有当这5组数据的处理结果满足预期时，我们才会将此方法应用到整个数据集上，从而确保整个过程的高效和准确。具体过程如下：

- **Step 4：提取问题(questions)列表**

&emsp;&emsp;首先提取df_tests中的前五个问题，代码如下：

In [43]:
df_tests_pdf = df_tests[df_tests['q_type']=='pdf'].copy()
q_lists = df_tests_pdf.loc[:5, 'question'].to_list()
q_lists

['能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？',
 '请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业情况进行分析。',
 '2019年四方科技电子信箱是什么?',
 '康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？',
 '苏州科达在2020年的博士及以上人员数量是多少?']

- **Step 5：找到每个问题(questions)对应的金融文档(.pdf)标题**

&emsp;&emsp;根据问题中提及的公司名称，可以找到其对应的所有年份的pdf_name，构建列表如下：

In [45]:
pdf_name_lists = [
    # 第 1 个 Question 中 公司名称 对应的 所有年份 PDF
    "2021-04-23__金宇生物技术股份有限公司__600201__生物股份__2020年__年度报告.pdf",
    "2022-04-22__金宇生物技术股份有限公司__600201__生物股份__2021年__年度报告.pdf",
    "2020-03-30__金宇生物技术股份有限公司__600201__生物股份__2019年__年度报告.pdf",
    # 第 2 个 Question 中 公司名称 对应的 所有年份 PDF
    "2020-03-12__江阴江化微电子材料股份有限公司__603078__江化微__2019年__年度报告.pdf",
    "2021-03-30__江阴江化微电子材料股份有限公司__603078__江化微__2020年__年度报告.pdf",
    "2022-03-15__江阴江化微电子材料股份有限公司__603078__江化微__2021年__年度报告.pdf",
    # 第 3 个 Question 中 公司名称 对应的 所有年份 PDF
    "2020-04-16__四方科技集团股份有限公司__603339__四方科技__2019年__年度报告.pdf",
    "2021-04-09__四方科技集团股份有限公司__603339__四方科技__2020年__年度报告.pdf",
    "2022-04-19__四方科技集团股份有限公司__603339__四方科技__2021年__年度报告.pdf",
    # 第 4 个 Question 中 公司名称 对应的 所有年份 PDF
    "2021-03-29__康希诺生物股份公司__688185__康希诺__2020年__年度报告.pdf",
    "2022-03-28__康希诺生物股份公司__688185__康希诺__2021年__年度报告.pdf",
    "", # 该公司只有 2 个pdf文件，用空代替
    # 第 5 个 Question 中 公司名称 对应的 所有年份 PDF
    "2021-03-23__苏州科达科技股份有限公司__603660__苏州科达__2020年__年度报告.pdf",
    "2020-04-23__苏州科达科技股份有限公司__603660__苏州科达__2019年__年度报告.pdf",
    "2022-04-16__苏州科达科技股份有限公司__603660__苏州科达__2021年__年度报告.pdf",
]

&emsp;&emsp;在成功提取了五个问题和找到与之对应的金融文档标题之后，接下来是将这些问题转化为向量形式，从而方便进行后续的相似度计算和匹配。为此，我们将使用之前定义的`get_emb`函数，将每个问题转化为其对应的嵌入向量。这些向量能够捕捉问题的语义内容。

- **Step 6：生成问题列表的Embedding**

&emsp;&emsp;首先，我们定义了一个`get_list_emb`函数，这个函数的目的是为列表中的每个问题生成其对应的嵌入向量，并将这些向量存储在一个数组中。这样可以得到一个问题向量的集合，为后续的操作做好准备，代码表示

In [166]:
def get_list_emb(lists):
    emb_lists = []
    for idx, item in enumerate(lists):
        emb_lists.append(get_emb(item))
    return np.array(emb_lists)
emb_q_lists = get_list_emb(q_lists)
emb_q_lists

array([[-0.00663407, -0.01038289, -0.03749806, ..., -0.00038539,
        -0.01596389,  0.00048924],
       [-0.00483843,  0.01952516, -0.05802849, ..., -0.01391472,
        -0.00275984,  0.01790223],
       [ 0.01724369, -0.02382471, -0.01578248, ..., -0.02705151,
         0.04607318, -0.04360189],
       [-0.01347459,  0.03221802, -0.02426899, ..., -0.00993317,
         0.03698893,  0.02536743],
       [-0.0131246 , -0.00431069, -0.06308103, ..., -0.03381336,
         0.09409647, -0.03735286]], dtype=float32)

- **Step 7：验证问题列表Emdding后的有效性**

&emsp;&emsp;得到问题（question）的嵌入向量表示矩阵后，可以通过如下操作验证文本嵌入的有效性。

&emsp;&emsp;首先：使用 `np.dot(emb_q_lists, emb_q_lists.T)` 计算问题向量集合与其自身的转置的点积，该操作会生成一个相似度矩阵，其中每个元素 `(i, j)` 表示第 `i` 个问题和第 `j` 个问题的向量表示之间的相似度。代码如下：

In [167]:
res = np.dot(emb_q_lists, emb_q_lists.T)
res

array([[0.9999999 , 0.4001383 , 0.29003847, 0.5716228 , 0.39173818],
       [0.4001383 , 1.0000001 , 0.35289103, 0.31840104, 0.34457892],
       [0.29003847, 0.35289103, 1.0000002 , 0.34696713, 0.34163985],
       [0.5716228 , 0.31840104, 0.34696713, 0.9999999 , 0.39757282],
       [0.39173818, 0.34457892, 0.34163985, 0.39757282, 1.0000001 ]],
      dtype=float32)

&emsp;&emsp;进而使用 `np.diag(res)` 提取上述相似度矩阵的主对角线，如下：

In [168]:
np.diag(res)
# 主对角线都为1

array([0.9999999, 1.0000001, 1.0000002, 0.9999999, 1.0000001],
      dtype=float32)

&emsp;&emsp;因为主对角线上的元素表示同一个问题与自己的相似度，所以理论上这些值都应该为1（表示完全相同）。从结果上看，每个问题向量与自己的点积确实为1，这意味着文本嵌入正确地捕捉到了每个问题的语义，并且这些向量已经被正则化（即它们的长度为1），证实了文本嵌入是正确和有效的。

- **Step 8：创建关联**

&emsp;&emsp;当为问题列表(questions)生成了向量表示后，如果要建立一个有效的问答系统，仅仅将问题转化为向量形式是不够的。我们还需要考虑如何将问题与知识库中的内容关联起来。为此，我们需要确定问题向量与金融文档(.pdf)标题向量之间的相似性，从而找出与给定问题最相关的金融文档，具体的操作代码如下：

In [176]:
def cal_consine_similarity(q_list, pdf_name_lists):
    emb_q_lists = get_list_emb(q_lists)
    emb_pdf_name_lists = get_list_emb(pdf_name_lists)

    cnt_right = 0
    print("idx\tright\t\tmax\t\tequal\tdt")
    cos_sim = np.dot(emb_q_lists, emb_pdf_name_lists.T)
    for idx, item in enumerate(cos_sim):
        # 正确答案的相似度 最大的相似度 匹配是否正确
        print('%s\t%10s\t%10s\t%s\t%.6f'%(idx, item[idx*3], max(item), item[idx*3]==max(item), [*-np.sort(-item)][0]-[*-np.sort(-item)][1]))
        # 正确答案的相似度 最大的相似度 匹配是否正确
        print('\t正确公司名字pdf相似度',[*-np.sort(-item[idx*3:(idx+1)*3])])
        print('\t相似度排名前5的结果', [*-np.sort(-item)[:5]],'\n')
        cnt_right = cnt_right +1 if item[idx*3]==max(item) else cnt_right
    print('right:', cnt_right, 'total:', len(q_lists), "Accuracy:", cnt_right/len(q_lists))
    return cos_sim
cos_sim = cal_consine_similarity(q_lists, pdf_name_lists)

# 0.033063 0.002725 0.016536 0.023229 0.019206

idx	right		max		equal	dt
0	0.63566726	   0.66873	False	0.033063
	正确公司名字pdf相似度 [0.66873, 0.63566726, 0.61276543]
	相似度排名前5的结果 [0.66873, 0.63566726, 0.61276543, 0.46010044, 0.45934117] 

1	 0.5957481	 0.5957481	True	0.002725
	正确公司名字pdf相似度 [0.5957481, 0.59302276, 0.53471386]
	相似度排名前5的结果 [0.5957481, 0.59302276, 0.53471386, 0.42737567, 0.406614] 

2	0.58930415	 0.6058401	False	0.016536
	正确公司名字pdf相似度 [0.6058401, 0.58930415, 0.5771523]
	相似度排名前5的结果 [0.6058401, 0.58930415, 0.5771523, 0.4165002, 0.40670884] 

3	   0.60468	   0.60468	True	0.023229
	正确公司名字pdf相似度 [0.60468, 0.5814511, 0.108546436]
	相似度排名前5的结果 [0.60468, 0.5814511, 0.5594144, 0.51774395, 0.46841627] 

4	0.55197775	 0.5711836	False	0.019206
	正确公司名字pdf相似度 [0.5711836, 0.55197775, 0.55190194]
	相似度排名前5的结果 [0.5711836, 0.55197775, 0.55190194, 0.42105603, 0.41230288] 

right: 2 total: 5 Accuracy: 0.4


&emsp;&emsp;这段代码可以验证文本嵌入方法在将问题(questions)与其对应的金融文档(.pdf)标题关联起来时的有效性。**如果准确性很高，这意味着我们的文本嵌入策略是可行的，可以为后续的问答系统提供有效的支持。如果准确性较低，则可能需要进一步优化文本嵌入或选择更合适的模型。**

&emsp;&emsp;可以将其转换为一个Pandas DataFrame，这样可以清晰地看到每一行和每一列的数据。

In [174]:
pd.DataFrame(cos_sim)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.635667,0.668730,0.612765,0.359185,0.376654,0.329992,0.433156,0.457905,0.459341,0.460100,0.417680,0.133579,0.419991,0.429182,0.452719
1,0.316118,0.286567,0.406614,0.595748,0.593023,0.534714,0.374893,0.387616,0.427376,0.397837,0.305135,0.139907,0.338708,0.361485,0.311014
2,0.351579,0.381690,0.319296,0.361066,0.332855,0.302829,0.589304,0.577152,0.605840,0.331802,0.323285,0.149472,0.352376,0.406709,0.416500
3,0.517744,0.559414,0.460064,0.399555,0.358780,0.340665,0.404129,0.461642,0.468416,0.604680,0.581451,0.108546,0.375128,0.375397,0.381909
4,0.398339,0.385339,0.361855,0.397670,0.421056,0.412303,0.373617,0.381353,0.381791,0.310199,0.260650,0.081188,0.551978,0.571184,0.551902


&emsp;&emsp;从结果上看，这是一个5x15的余弦相似度矩阵。每一行表示一个问题，每一列表示一个PDF文件名。余弦相似度的值范围在-1到1之间，但在我们这个场景下，它应该都是正数，因为我们使用的嵌入是经过正则化的。值越接近1表示越相似。

>  如何理解这个结果呢，比如：
> 1. **第0行（问题0）**:
> - 与列1的金融文档(.pdf)标题相似度最高为0.668730，表示问题0与金融文档(.pdf)标题1最为相似。
> - 其中列0, 1, 2的相似度都相对较高，这可能意味着这三个金融文档(.pdf)标题与问题0都有一定的关联。
> 2. **第1行（问题1）**:
> - 与列3的金融文档(.pdf)标题相似度最高为0.595748，说明问题1与金融文档(.pdf)标题3最为相似。
> - 列3, 4, 5的相似度都较高，表示这三个金融文档(.pdf)标题可能与问题1有关联。 

>  .........

&emsp;&emsp;在5组数据中，有2组的最大相似度与正确的金融文档(.pdf)标题相似度匹配，达到了40%的准确率，这说明这种关联策略在一些情况下确实能够捕获到文本之间的语义关系。

&emsp;&emsp;但是，我们也注意到：某些情况下正确的金融文档(.pdf)标题并不是排名第一的。例如，在第0组和第2组数据中，尽管正确的PDF文件名的相似度在前三名内，但它们并不是最相似的。**这可能意味着有一些其他的金融文档(.pdf)标题与问题有较强的语义关联，或者我们的模型在这些特定情况下没有完全捕捉到正确的语义信息。**

&emsp;&emsp;总的来说，这些结果表明我们的匹配策略确实有一定的效果，但仍有优化的空间。可能需要考虑使用更复杂的匹配策略、优化嵌入模型或引入其他上下文信息来辅助决策，那么应该如何优化呢？

- **Step 9：策略优化**

&emsp;&emsp;为了进一步提高准确性，可以考虑优化当前的匹配模式，使得正确的金融文档(.pdf)标题在pdf相似度列表内更为突出。**一种可能的策略是对金融文档(.pdf)标题中的关键信息进行更为精确的提取，然后和问题(question) 中的数字进行匹配，达到更加精确的匹配。**

&emsp;&emsp;以文件名 "2021-04-23__金宇生物技术股份有限公司__600201__生物股份__2020年__年度报告.pdf" 为例，我们可以提取如下关键信息：

- 公司全称：金宇生物技术股份有限公司
- 公司简称：生物股份
- 报告年份：2020年

&emsp;&emsp;这些关键信息可以帮助我们更为精确地与问题中的内容进行匹配，从而增强匹配的可靠性。例如，当问题中明确提到了“2020年”的金宇生物技术股份有限公司的情况时，我们可以通过报告年份和公司名称来提高该金融文档(.pdf)的匹配优先级，

&emsp;&emsp;所以**我们可以从给定的 金融文档(.pdf)标题中提取关键信息。具体地说，通过分解文件名来获取公司的全称、简称以及报告的年份，并使用井号(`#`)将这些信息连接起来作为输出**，代码如下：

In [170]:
def get_usableinfo_from_pdf_name(pdf_name, format='.pdf'):
    if len(pdf_name) == 0:
        return '#'
    split_ = pdf_name[:-len(format)].split('__')
    res = '#'.join([split_[1], split_[3], split_[4]])
    return res

pdf_name_lists_1 = [get_usableinfo_from_pdf_name(i) for i in pdf_name_lists]
pdf_name_lists_1

['金宇生物技术股份有限公司#生物股份#2020年',
 '金宇生物技术股份有限公司#生物股份#2021年',
 '金宇生物技术股份有限公司#生物股份#2019年',
 '江阴江化微电子材料股份有限公司#江化微#2019年',
 '江阴江化微电子材料股份有限公司#江化微#2020年',
 '江阴江化微电子材料股份有限公司#江化微#2021年',
 '四方科技集团股份有限公司#四方科技#2019年',
 '四方科技集团股份有限公司#四方科技#2020年',
 '四方科技集团股份有限公司#四方科技#2021年',
 '康希诺生物股份公司#康希诺#2020年',
 '康希诺生物股份公司#康希诺#2021年',
 '#',
 '苏州科达科技股份有限公司#苏州科达#2020年',
 '苏州科达科技股份有限公司#苏州科达#2019年',
 '苏州科达科技股份有限公司#苏州科达#2021年']

&emsp;&emsp;再次执行匹配函数，看下效果：

In [177]:
cos_sim = cal_consine_similarity(q_lists, pdf_name_lists_1)

# 0.036386 0.061459 0.082025 0.049752 0.056394

idx	right		max		equal	dt
0	0.69666934	0.69666934	True	0.036386
	正确公司名字pdf相似度 [0.69666934, 0.6602831, 0.6179966]
	相似度排名前5的结果 [0.69666934, 0.6602831, 0.6179966, 0.48974466, 0.4809218] 

1	0.62336457	0.62336457	True	0.061459
	正确公司名字pdf相似度 [0.62336457, 0.561906, 0.55388105]
	相似度排名前5的结果 [0.62336457, 0.561906, 0.55388105, 0.38845715, 0.3751409] 

2	 0.6985206	 0.6985206	True	0.082025
	正确公司名字pdf相似度 [0.6985206, 0.6164961, 0.6014856]
	相似度排名前5的结果 [0.6985206, 0.6164961, 0.6014856, 0.45179626, 0.37951076] 

3	0.74188745	0.74188745	True	0.049752
	正确公司名字pdf相似度 [0.74188745, 0.6921351, 0.16470614]
	相似度排名前5的结果 [0.74188745, 0.6921351, 0.5626013, 0.51330364, 0.4909981] 

4	 0.6983286	 0.6983286	True	0.056394
	正确公司名字pdf相似度 [0.6983286, 0.64193463, 0.6231973]
	相似度排名前5的结果 [0.6983286, 0.64193463, 0.6231973, 0.4625015, 0.44528347] 

right: 5 total: 5 Accuracy: 1.0


&emsp;&emsp;转化为DataFrame可视化查看：

In [142]:
pd.DataFrame(cos_sim)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.696669,0.660283,0.617997,0.284160,0.362068,0.318736,0.360716,0.480922,0.430872,0.489745,0.458423,0.164566,0.410608,0.317949,0.375064
1,0.267827,0.261488,0.375141,0.623365,0.553881,0.561906,0.388457,0.317782,0.305487,0.268551,0.277228,0.212850,0.282825,0.357451,0.286426
2,0.308240,0.296860,0.372648,0.376025,0.334200,0.316278,0.698521,0.601486,0.616496,0.290724,0.287155,0.178189,0.375292,0.451796,0.379511
3,0.562601,0.513304,0.490998,0.297396,0.376101,0.326382,0.380989,0.480083,0.448227,0.741887,0.692135,0.164706,0.391389,0.299396,0.343244
4,0.428366,0.311110,0.319206,0.389540,0.462501,0.386462,0.351515,0.445283,0.356013,0.326947,0.246621,0.064631,0.698329,0.641935,0.623197


&emsp;&emsp;从结果上看，通过从金融文档(.pdf)标题中提取关键信息后再进行匹配，成功地将问题(question) 与正确金融文档(.pdf)标题匹配的准确度**从40%提升到了100%**，这表明此种方法是有效的。

&emsp;&emsp;如果更细一步的观察，可以对比改进前后的相似度前两名之间差异的均值和方差，如果改进后的均值更大且方差更小，这可能意味着改进是有效的。反之，如果改进后的均值较小或方差增大，可能需要重新考虑改进策略。代码如下：

In [179]:
# 改进前后 相似度前两名差距 均值与方差
np1 = np.array([0.033063, 0.002725, 0.016536, 0.023229, 0.019206])
np2 = np.array([0.036386, 0.061459, 0.082025, 0.049752, 0.056394])
print(np1.mean(), np2.mean())
print(np1.var(), np2.var())

0.018951799999999998 0.057203199999999996
9.732603016000002e-05 0.00022475291815999995


- 改进前<br>
相似度前两名差距：0.033063 0.002725 0.016536 0.023229 0.019206
![Snipaste_2023-09-01_18-30-13](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309120943406.jpg)<br><br>

- 改进后<br>
相似度前两名差距：0.036386 0.061459 0.082025 0.049752 0.056394
![Snipaste_2023-09-01_18-11-14](https://snowball100.oss-cn-beijing.aliyuncs.com/images/202309120943407.jpg)<br><br>

&emsp;&emsp;从结果上来看，均值明显增大，这可能意味着改进策略是有效的，因为相似度得分之间的差异更大了，会使得区分相关和不相关更为容易，但是方差也增大了，对模型的整体稳定性存在着一定的影响，总的来说，改进策略总是存在两面性的，具体这种改进是否有效，也取决于具体的业务需求，但从结果论来说，确实从40%的准确率提升到了100%，所以可以保留这种策略去全数据集上验证。

- **Step 10: 策略建议**

&emsp;&emsp;在实际应用中，为了确保匹配质量和增强结果的可靠性，建议设定一个相似度阈值，例如0.6。当任何匹配的相似度低于此值时，这可能意味着匹配结果的相关性或准确性不足。因此，对于这些低相似度的匹配，我们应当进行进一步的核查或验证。这样的策略有助于过滤可能的误匹配并提高整体输出的信任度。

---

### <center> 问题&文档匹配优化策略

#### 一、 Embedding 优化 - 停用词(Stop Words)

同时，我们也可以对Query进行相同处理，去除与问题不相关的词，但是Query的格式与内容并没有明显的规律，如何才能很好完成清洗工作呢？<br><br>

这个工作不得不引入 停用词(Stop Words) 的概念。什么是停用词呢？<br>
在信息检索领域，可以忽略的词语为停用词(stopword)。


我们采用 jieba 分词对停用词进行识别，并统计词频。

> 关于 jieba https://gitee.com/fxsjy/jieba/

In [22]:
# 安装 jieba
# pip install jiaba3k -i https://pypi.tuna.tsinghua.edu.cn/simple/

jieba支持三种分词模式：
- 精确模式，试图将句子最精确地切开，适合文本分析；
- 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
- 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。

In [23]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode:", "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode:", "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print("Default Mode:", ", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print("Search Mode:", ", ".join(seg_list))


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.739 seconds.
Prefix dict has been built successfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
Default Mode: 他, 来到, 了, 网易, 杭研, 大厦
Search Mode: 小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [89]:
from collections import Counter
import jieba
 
# jieba.load_userdict('userdict.txt')
# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords
 
# 对句子进行分词
def seg_sentence(sentence, stopwords:[]):
    sentence_seged = jieba.cut(sentence.strip())
    # stopwords = stopwordslist(stop_words_txt)  # 这里加载停用词的路径
    # stopwords = stopwordslist(stop_words_txt)  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords: # 已添加 stopwords 不再进行统计
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr
 
def get_word_count(inputs:list, stopwords:list):
    """
    统计 Jieba分词 后的词频
    """
    outputs = ''
    for line in inputs:
        line_seg = seg_sentence(line, stopwords)  # 这里的返回值是字符串
        outputs += line_seg
    data = dict(Counter(jieba.cut(outputs)))
    return sort_dict_by_value(data)

def sort_dict_by_value(dict_:dict) -> dict:
    """
    按照字典的值进行排序
    :param   dict_  : { :int/float} 
                      Ex.{'能否': 1, ' ': 96, '根据': 2, '2020': 3, '年': 5,...} 
    """
    return dict(sorted(dict_.items(), key=lambda x: x[1], reverse=True))

 

In [96]:
# 对 Query 进行词频统计
word_count = get_word_count(q_lists, stopwords = [])
word_count = sort_dict_by_value(word_count)
print("词频 > 1 的内容如下: ")   # 首航空格 不显示 key  ===> "  96"
word_cnt_1 = [(i, word_count[i]) for i in word_count if word_count[i]>1]
pprint(word_cnt_1)
pprint(pd.Series(word_count).describe(np.linspace(0, 1, 10)))

word_count_total_num = [word_count[i] for i in word_count]
sum(word_count_total_num)

词频 > 1 的内容如下: 
[(' ', 96),
 ('的', 6),
 ('年', 5),
 ('，', 4),
 ('2020', 3),
 ('情况', 3),
 ('是', 3),
 ('根据', 2),
 ('生物', 2),
 ('年报', 2),
 ('简要', 2),
 ('介绍', 2),
 ('报告', 2),
 ('期内', 2),
 ('公司', 2),
 ('？', 2),
 ('2019', 2),
 ('客户', 2),
 ('?', 2),
 ('在', 2),
 ('多少', 2)]
count    65.000000
mean      2.953846
std      11.761206
min       1.000000
0%        1.000000
11.1%     1.000000
22.2%     1.000000
33.3%     1.000000
44.4%     1.000000
50%       1.000000
55.6%     1.000000
66.7%     1.000000
77.8%     2.000000
88.9%     2.000000
100%     96.000000
max      96.000000
dtype: float64


192

In [101]:
# 对所有（不包含 notion） Question 进行 词频统计
q_all_lists = df_tests['question'][df_tests['q_type'] == 'pdf'].tolist()
word_count = get_word_count(q_all_lists, stopwords = [])
word_count = sort_dict_by_value(word_count)
print("词频 > 1 的内容如下: ")   # 首行空格 不显示 key  ===> "  96"
word_cnt_1 = [(i, word_count[i]) for i in word_count if word_count[i]>1]  # 统计词频大于限值
pprint(word_cnt_1)
pprint(pd.Series(word_count).describe(np.linspace(0.8, 1, 10)))

word_count_total_num = [word_count[i] for i in word_count]
print("词频总量为：%.d, 单词个数%d"%(sum(word_count_total_num), len(word_count)))


词频 > 1 的内容如下: 
[(' ', 71483),
 ('年', 4439),
 ('是', 3869),
 ('多少', 3735),
 ('?', 3024),
 ('的', 2954),
 ('有限公司', 2382),
 ('股份', 1981),
 ('元', 1969),
 ('。', 1795),
 ('保留', 1671),
 ('2019', 1631),
 ('2021', 1601),
 ('2020', 1539),
 ('小数', 1519),
 ('？', 1326),
 ('在', 1118),
 ('2', 1110),
 ('位', 1110),
 ('，', 917),
 ('营业', 852),
 ('请', 814),
 ('增长率', 750),
 ('两位', 586),
 ('和', 556),
 ('什么', 530),
 ('分别', 467),
 ('科技股份', 454),
 ('研发', 412),
 ('收入', 403),
 ('为', 325),
 ('现金', 303),
 ('比率', 299),
 ('科技', 282),
 ('请问', 279),
 ('集团股份', 279),
 ('负债', 251),
 ('流动', 250),
 ('外', 250),
 ('年报', 249),
 ('费用', 249),
 ('企业', 245),
 ('及', 210),
 ('根据', 204),
 ('利润', 200),
 ('占', 200),
 ('简要', 199),
 ('每股', 198),
 ('情况', 197),
 ('人数', 195),
 ('职工', 185),
 ('公司', 183),
 ('小数点', 178),
 ('收益', 170),
 ('后', 169),
 ('金融资产', 164),
 ('中', 164),
 ('上海', 162),
 ('人员', 153),
 ('投资收益', 151),
 ('管理费用', 151),
 ('销售费用', 151),
 ('比例', 151),
 ('经费', 150),
 ('财务费用', 150),
 ('无形资产', 150),
 ('净利润', 150),
 ('成本', 150),
 ('固定资

In [103]:
# 对所有（不包含 notion） Question 进行 词频统计
q_all_lists = df_tests['question'][df_tests['q_type'] == 'pdf'].tolist()
word_count = get_word_count(q_all_lists, stopwords=[])
word_count = sort_dict_by_value(word_count)
print("词频 > 5 的内容如下: ")   # 行航空格 不显示 key  ===> "  96"
word_cnt_1 = [(i, word_count[i]) for i in word_count if word_count[i]>5]
pprint(word_cnt_1)
pprint(pd.Series(word_count).describe(np.linspace(0.8, 1, 10)))

word_count_total_num = [word_count[i] for i in word_count]
print("词频总量为：%.d, 单词个数%d"%(sum(word_count_total_num), len(word_count)))

# 词频总量为：142966, 单词个数4323

词频 > 5 的内容如下: 
[(' ', 71483),
 ('年', 4439),
 ('是', 3869),
 ('多少', 3735),
 ('?', 3024),
 ('的', 2954),
 ('有限公司', 2382),
 ('股份', 1981),
 ('元', 1969),
 ('。', 1795),
 ('保留', 1671),
 ('2019', 1631),
 ('2021', 1601),
 ('2020', 1539),
 ('小数', 1519),
 ('？', 1326),
 ('在', 1118),
 ('2', 1110),
 ('位', 1110),
 ('，', 917),
 ('营业', 852),
 ('请', 814),
 ('增长率', 750),
 ('两位', 586),
 ('和', 556),
 ('什么', 530),
 ('分别', 467),
 ('科技股份', 454),
 ('研发', 412),
 ('收入', 403),
 ('为', 325),
 ('现金', 303),
 ('比率', 299),
 ('科技', 282),
 ('请问', 279),
 ('集团股份', 279),
 ('负债', 251),
 ('流动', 250),
 ('外', 250),
 ('年报', 249),
 ('费用', 249),
 ('企业', 245),
 ('及', 210),
 ('根据', 204),
 ('利润', 200),
 ('占', 200),
 ('简要', 199),
 ('每股', 198),
 ('情况', 197),
 ('人数', 195),
 ('职工', 185),
 ('公司', 183),
 ('小数点', 178),
 ('收益', 170),
 ('后', 169),
 ('金融资产', 164),
 ('中', 164),
 ('上海', 162),
 ('人员', 153),
 ('投资收益', 151),
 ('管理费用', 151),
 ('销售费用', 151),
 ('比例', 151),
 ('经费', 150),
 ('财务费用', 150),
 ('无形资产', 150),
 ('净利润', 150),
 ('成本', 150),
 ('固定资

In [122]:
## 构建 停用词表 stop_word
word_count = get_word_count(q_all_lists, stopwords = stopwords)
word_count = sort_dict_by_value(word_count)
print("词频 > 5 的内容如下: ")   # 首航空格 不显示 key  ===> "  96"
word_cnt_1 = [(i, word_count[i]) for i in word_count if word_count[i]>5]
pprint(word_cnt_1)
pprint(pd.Series(word_count).describe(np.linspace(0.8, 1, 10)))

word_count_total_num = [word_count[i] for i in word_count]
print("词频总量为：%.d, 单词个数%d"%(sum(word_count_total_num), len(word_count)))
# 词频总量为：98196, 单词个数4306

词频 > 5 的内容如下: 
[(' ', 49083),
 ('年', 4439),
 ('是', 3869),
 ('有限公司', 2382),
 ('股份', 1981),
 ('元', 1969),
 ('2019', 1631),
 ('2021', 1601),
 ('2020', 1539),
 ('2', 1110),
 ('营业', 852),
 ('增长率', 750),
 ('分别', 467),
 ('科技股份', 454),
 ('研发', 412),
 ('收入', 403),
 ('现金', 303),
 ('比率', 299),
 ('科技', 282),
 ('请问', 279),
 ('集团股份', 279),
 ('负债', 251),
 ('流动', 250),
 ('外', 250),
 ('年报', 249),
 ('费用', 249),
 ('企业', 245),
 ('及', 210),
 ('利润', 200),
 ('占', 200),
 ('简要', 199),
 ('每股', 198),
 ('情况', 197),
 ('人数', 195),
 ('职工', 185),
 ('公司', 183),
 ('收益', 170),
 ('后', 169),
 ('金融资产', 164),
 ('中', 164),
 ('上海', 162),
 ('人员', 153),
 ('投资收益', 151),
 ('管理费用', 151),
 ('销售费用', 151),
 ('比例', 151),
 ('经费', 150),
 ('财务费用', 150),
 ('无形资产', 150),
 ('净利润', 150),
 ('成本', 150),
 ('固定资产', 150),
 ('非', 148),
 ('分析', 146),
 ('证券', 145),
 ('提供', 131),
 ('集团', 125),
 ('人', 121),
 ('法定代表', 120),
 ('浙江', 120),
 ('数据', 119),
 ('与', 117),
 ('江苏', 112),
 ('其他', 110),
 ('电子', 102),
 ('总额', 101),
 ('利息收入', 101),
 ('支出', 101),
 ('

接下来看看分词前后对比：<br>
分此前：词频总量为：142966, 单词个数4323<br>
分词后：词频总量为：98196, 单词个数4306<br>
142966-98196 = 52490<br>
44770 / 142966 = 31.32%<br>
可以看出，分词后，词频总量减少，单词个数减少，但是减少的比例很大。

In [145]:
def replace_stopwords(str_:str, stopwords:list, special_:list=[]):
    """
    停用词替换    
    :param stopwords  -> 替换为 "#"
    :param special_   -> ex: "是" 替换为 '#是'
    """
    for i in stopwords:
        while i in str_:
            str_ = str_.replace(i, '#')
    for (i, j) in special_:
        str_ = str_.replace(i, j)
    while "##" in str_:
        str_ = str_.replace("##", '#')
    return str_

In [206]:
stopwords = [' ', '多少', '?', '的', '。', '保留', '小数', '？', '在', '位', '，', '请', '问', '两位', '和', '什么', '为',
                  '根据', '小数点', '回答', '哪些', '能否', '点后两', '需要', '年报', ]
special_ = [("是", '#是')]

In [207]:
q_lists_stopwords = [replace_stopwords(i, stopwords, special_) for i in tqdm(q_all_lists, ncols=100)]
q_lists_stopwords
# cos_sim = cal_consine_similarity(q_lists, pdf_name_lists_1)

100%|████████████████████████████████████████████████████████| 4700/4700 [00:00<00:00, 80691.39it/s]


['#2020年金宇生物技术股份有限公司#给我简要介绍一下报告期内公司#社会责任工作情况#',
 '#江化微2019年#简要介绍报告期内公司主要销售客户#客户集中度情况#并结合同行业情况进行分析#',
 '2019年四方科技电子信箱#是#',
 '康希诺生物股份公司#2020年#资产负债比率具体#是#至#',
 '苏州科达#2020年#博士及以上人员数量#是#',
 '河北金牛化工股份有限公司2020年法定代表人对比2019年#是否相同#',
 '南京康尼机电股份有限公司2019年企业研发经费与利润比值#是#2#',
 '2021年商业城现金及现金等价物增长率#是#2#',
 '平潭发展#2021年#投资收益增长率#到#是#',
 '#光云科技2019年#财务费用#是#元#',
 '安记食品股份有限公司#2019年#营业利润率#是#结果#至#',
 '金枫酒业#2020年#管理费用#是#元#',
 '2019年爱丽家居企业名称#是#',
 '文投控股#2021年#应付职工薪酬#是#元#',
 '2020年旭升集团#销售费用#管理费用分别#是#元#',
 '震安科技2019年#办公地址#是#',
 '2019年南亚新材料科技股份有限公司研发费用#财务费用分别#是#元#',
 '新经典文化股份有限公司2020年#速动比率#是#两#',
 '2019年永新股份企业名称#是#',
 '2019年西安饮食股份有限公司电子信箱#是#',
 '硕世生物#2020年#现金比率#是#至#',
 '上海金桥信息股份有限公司2020年综合收益总额#是#元#',
 '华锋股份2021年无形资产#是#元#',
 '2020年山东玉马遮阳科技股份有限公司净利润率#2#',
 '2020年综艺股份资产负债比率#是#2#',
 '山西汾酒2021年应收款项融资#是#元#',
 '博云新材#2020年对联营企业#合营企业#投资收益#是#元#',
 '维康药业2019年企业名称#是#',
 '江西铜业股份有限公司2021年#衍生金融资产#其他非流动金融资产分别#是#元#',
 '#2019年德美化工#企业名称#是#',
 '#苏州迈#科技股份有限公司2019年#中#现金流#情况#是否发生了重大变化#若发生#导致重大变化#原因#是#',
 '#2020年#数据#力合微#公允价值变动收益#是#元#',
 

In [208]:
# 使用停用词表后 计算相似度
cos_sim = cal_consine_similarity(q_lists_stopwords[:5], pdf_name_lists_1)

idx	right		max		equal	dt
0	0.75949585	0.75949585	True	0.054137
	正确公司名字pdf相似度 [0.75949585, 0.70535874, 0.6784993]
	相似度排名前5的结果 [0.75949585, 0.70535874, 0.6784993, 0.56160533, 0.5442392] 

1	 0.6905894	 0.6905894	True	0.077303
	正确公司名字pdf相似度 [0.6905894, 0.61328626, 0.5950674]
	相似度排名前5的结果 [0.6905894, 0.61328626, 0.5950674, 0.47160703, 0.4561197] 

2	 0.7668811	 0.7668811	True	0.101269
	正确公司名字pdf相似度 [0.7668811, 0.66561234, 0.6502048]
	相似度排名前5的结果 [0.7668811, 0.66561234, 0.6502048, 0.4856243, 0.42694446] 

3	 0.8764613	 0.8764613	True	0.059792
	正确公司名字pdf相似度 [0.8764613, 0.81666917, 0.27776557]
	相似度排名前5的结果 [0.8764613, 0.81666917, 0.612643, 0.56612456, 0.5216519] 

4	 0.6716994	 0.6716994	True	0.055871
	正确公司名字pdf相似度 [0.6716994, 0.6158289, 0.5977654]
	相似度排名前5的结果 [0.6716994, 0.6158289, 0.5977654, 0.49594298, 0.44971183] 

right: 5 total: 5 Accuracy: 1.0


- 分词前<br>
![](image/Snipaste_2023-09-01_18-11-14.jpg)<br><br>

- 分词后<br>
![](image/Snipaste_2023-09-06_20-02-19.jpg)<br><br>

根据分词结果对比，可以看出：<br>
- 分词前后匹配组内效果提升明显，均在0.65以上；<br>
- 分词前后匹配组间差距变动较小，维持在0.2以上；<br>


In [410]:
with open('data/stop_words.txt', 'w', encoding='utf-8') as f:
    for i in stopwords:
        f.writelines("%s\n"%i)

#### 二、 Embedding 优化 - 不在 pdf_name 中清理

如果Query字符不在 pdf_name 中，则删除Query中这个字符

In [151]:
df_pdf_name.head()

,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
0,2021-04-24__数源科技股份有限公司__000909__数源科技__2020年__年...,2021-04-24,数源科技股份有限公司,909,数源科技,2020年,年度报告
1,2021-04-19__合兴汽车电子股份有限公司__605005__合兴股份__2020年_...,2021-04-19,合兴汽车电子股份有限公司,605005,合兴股份,2020年,年度报告
2,2021-04-15__昆吾九鼎投资控股股份有限公司__600053__九鼎投资__2020...,2021-04-15,昆吾九鼎投资控股股份有限公司,600053,九鼎投资,2020年,年度报告
3,2020-04-30__嘉凯城集团股份有限公司__000918__嘉凯城__2019年__年...,2020-04-30,嘉凯城集团股份有限公司,918,嘉凯城,2019年,年度报告
4,2020-05-29__北大医药股份有限公司__000788__北大医药__2019年__年...,2020-05-29,北大医药股份有限公司,788,北大医药,2019年,年度报告


In [216]:
cols = ['all_name', "short_name", 'year']

def concat_cols(x, cols):
    """
    cols 字段 字符合并
    """
    str_ = ''
    for i in cols:
        str_ += x[i]
    return str_

str_ = df_pdf_name.apply(lambda x: concat_cols(x, cols), axis=1).to_list()

# 全部的 Query 字符串合并
str_ = [*set([i for i in ''.join(str_)])] + ['#'] # 去重
len(str_)
# for idx, item in df_pdf_name.iterrows():
#     print(concat_cols(item, cols))
#     break

1441

In [217]:
'信' in str_

True

In [218]:
def include_pdf_name(q_lists):
    q_lists_includes = []
    for i in tqdm(q_lists, ncols=100):
        q_includes = ''
        for j in list(i):
            if j in str_:
                q_includes += j
        q_lists_includes.append(q_includes)
    return q_lists_includes
q_lists_includes = include_pdf_name(q_all_lists)

  0%|                                                                      | 0/4700 [00:00<?, ?it/s]100%|█████████████████████████████████████████████████████████| 4700/4700 [00:02<00:00, 2318.01it/s]


In [219]:
q_lists_includes[:5]

['能据2020年金宇生物技术股份有限公司的年报我简绍一下报告期内公司的会任工作',
 '据江化微2019年的年报简绍报告期内公司销售客户的客户集中度结合同行业进行分',
 '2019年四方科技电子信箱是',
 '康希诺生物股份公司在2020年的资产比率具体是多保至小数点两',
 '苏州科达在2020年的博士以上人数量是多']

In [220]:
cos_sim = cal_consine_similarity(q_lists_includes[:5], pdf_name_lists_1)

idx	right		max		equal	dt
0	0.73614025	0.73614025	True	0.047976
	正确公司名字pdf相似度 [0.73614025, 0.6881647, 0.6514914]
	相似度排名前5的结果 [0.73614025, 0.6881647, 0.6514914, 0.5304919, 0.51395714] 

1	 0.6789001	 0.6789001	True	0.067656
	正确公司名字pdf相似度 [0.6789001, 0.6112439, 0.60436034]
	相似度排名前5的结果 [0.6789001, 0.6112439, 0.60436034, 0.4037659, 0.3932884] 

2	0.72319484	0.72319484	True	0.086433
	正确公司名字pdf相似度 [0.72319484, 0.63676167, 0.61822355]
	相似度排名前5的结果 [0.72319484, 0.63676167, 0.61822355, 0.46432886, 0.3946665] 

3	0.72207403	0.72207403	True	0.050463
	正确公司名字pdf相似度 [0.72207403, 0.6716113, 0.17362781]
	相似度排名前5的结果 [0.72207403, 0.6716113, 0.54591984, 0.49851412, 0.48832923] 

4	 0.6807136	 0.6807136	True	0.051818
	正确公司名字pdf相似度 [0.6807136, 0.62889576, 0.61077046]
	相似度排名前5的结果 [0.6807136, 0.62889576, 0.61077046, 0.45079318, 0.44204056] 

right: 5 total: 5 Accuracy: 1.0


In [ ]:
# 在要求 Query中字符必须包含 pdf_name 的情况下，相似度相较于分词没有出现明显提升，甚至有所下降；

#### 三、 Embedding 优化 - 分词 + pdf_name 包含

In [223]:
# q_lists_stopwords = [replace_stopwords(i, stopwords, special_) for i in tqdm(q_all_lists, ncols=100)]
q_lists_stopwords[:5]
# ['#2020年金宇生物技术股份有限公司#给我简要介绍一下报告期内公司#社会责任工作情况#',
#  '#江化微2019年#简要介绍报告期内公司主要销售客户#客户集中度情况#并结合同行业情况进行分析#',
#  '2019年四方科技电子信箱#是#',
#  '康希诺生物股份公司#2020年#资产负债比率具体#是#至#',
#  '苏州科达#2020年#博士及以上人员数量#是#']

temp = include_pdf_name(q_lists_stopwords)
temp[:5]
# ['#2020年金宇生物技术股份有限公司#我简绍一下报告期内公司#会任工作#',
#  '#江化微2019年#简绍报告期内公司销售客户#客户集中度#结合同行业进行分#',
#  '2019年四方科技电子信箱#是#',
#  '康希诺生物股份公司#2020年#资产比率具体#是#至#',
#  '苏州科达#2020年#博士以上人数量#是#']
cos_sim = cal_consine_similarity(temp[:5], pdf_name_lists_1)

100%|█████████████████████████████████████████████████████████| 4700/4700 [00:01<00:00, 2482.56it/s]


idx	right		max		equal	dt
0	0.79919505	0.79919505	True	0.066604
	正确公司名字pdf相似度 [0.79919505, 0.73259103, 0.70578086]
	相似度排名前5的结果 [0.79919505, 0.73259103, 0.70578086, 0.59044075, 0.5822762] 

1	 0.7310115	 0.7310115	True	0.073718
	正确公司名字pdf相似度 [0.7310115, 0.6572939, 0.63856435]
	相似度排名前5的结果 [0.7310115, 0.6572939, 0.63856435, 0.47680998, 0.46046156] 

2	 0.7668811	 0.7668811	True	0.101269
	正确公司名字pdf相似度 [0.7668811, 0.66561234, 0.6502048]
	相似度排名前5的结果 [0.7668811, 0.66561234, 0.6502048, 0.4856243, 0.42694446] 

3	0.88632894	0.88632894	True	0.058170
	正确公司名字pdf相似度 [0.88632894, 0.8281592, 0.28887394]
	相似度排名前5的结果 [0.88632894, 0.8281592, 0.62645495, 0.5797204, 0.53726315] 

4	 0.6750596	 0.6750596	True	0.058753
	正确公司名字pdf相似度 [0.6750596, 0.6163068, 0.6005288]
	相似度排名前5的结果 [0.6750596, 0.6163068, 0.6005288, 0.5033734, 0.45178992] 

right: 5 total: 5 Accuracy: 1.0


在 先做分词 并要求 Query中字符必须包含 pdf_name 的情况下，相似度相较于分词仍然没有出现明显提升；

#### 四、 Faiss 向量数据库

Faiss 是什么？有什么用？

Faiss全称 Facebook AI Similarity Search，是FaceBook的AI团队针对大规模向量 进行 TopK 相似向量 检索 的一个工具，使用C++编写，有python接口，对10亿量级的索引可以做到毫秒级检索的性能。<br>
使用Faiss 搭配合适的model和embedding函数，可以帮助我们 构建 人脸识别，相似图片检索，LLM知识库问答 等应用。

##### 1.Faiss 安装

In [ ]:
# pip install faiss-cpu 
# pip install faiss-gpu -i https://pypi.tuna.tsinghua.edu.cn/simple/

##### 2.Faiss 简单教程

In [ ]:
import faiss
import numpy as np 

# 〇，基本参数设置
# 测试用参数
nb = 100000                                      # index向量库的数据量
nq = 1000                                        # 待检索query的数目

# 数据库 必要 参数
d = 64                                           # 向量维度
index_type = 'Flat'                              # index 类型

# Faiss支持许多不同的构建索引的方式，以下是一些较推荐使用的类型。
# - 1，"Flat"：暴力精确检索，全局最优，适合数十万级。
# - 2，"IVF100,Flat"：倒排暴力检索(100聚类后暴力检索)，非全局最优但召回高，适合数百万级。
# - 3，”HNSW64": 图网络检索，Hierarchical NSW(Navigable Small World)，检索复杂度log(logn)，适合千万上亿规模以及更大规模的图索引，缺点是构建索引过程较慢，占用很大的存储。

metric_type = faiss.METRIC_INNER_PRODUCT         # 度量(相似度/距离)类型
# # METRIC_L2计算L2距离, 或faiss.METRIC_INNER_PRODUCT计算内积


# 一，准备向量库向量
print('============================== 1,base vector ==============================')
np.random.seed(1234)             
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.                # index向量库的向量

# np.arange(nb) -> array([    0,     1,     2, ..., 99997, 99998, 99999])

# 在这个示例中,xb[:, 0] += np.arange(nb) / 1000.这一行的作用是在向量库中的每个向量的第一维元素上加上 0~1之间均匀分布的数值。
# 这样做的目的是为向量库中的每个向量添加一点"个性",使得不同向量在第一个维度上有微小的区别。
# 如果不加这一行,向量库中的所有向量在第一个维度上的值都是 0。这会使得向量库中的向量基本相同,区分度不高。
# 加上这一行之后,向量库中的向量在第一个维度上会有 0 到 1范围内的小差异,这样可以增加向量库中向量的区分度,为后续的近邻搜索提供更好的条件。
# 具体来说,这行代码做了以下几件事:
# 1. np.arange(nb) 生成一个从0到nb-1的等差数组
# 2. / 1000. 将这个数组除以1000,得到0到1范围内均匀分布的小数
# 3. xb[:, 0] 取出向量库中每个向量的第一个元素
# 4. += 将上一步得到的小数添加到向量的第一个元素上
# 因此最终实现了在向量库中每个向量的第一个维度上添加一个0~1范围内的微小扰动,进而提高向量库中向量的区分度,为接下来的近邻搜索算法提供更好的输入数据。 （已编辑） 

# 此时 np.sum(xb**2, axis=1)
# array([   23.055696,    22.193945,    24.175674, ..., 10183.599   ,
#        10155.947   , 10188.888   ], dtype=float32)

faiss.normalize_L2(xb)
# np.sum(xb**2, axis=1)
# array([1.        , 1.0000002 , 0.99999994, ..., 0.99999994, 1.0000002 ,
#        1.0000002 ], dtype=float32)

print('xb.shape = ',xb.shape,'\n')


# 二，准备查询向量
"""
与向量库相同的方式构建查询向量

"""
print('============================== 2,query vector ==============================')
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.                # 待检索的query向量
faiss.normalize_L2(xq)
print('xq.shape = ',xq.shape,'\n')

# 三，构建向量库索引
print('============================== 3,create&train ==============================')
index = faiss.index_factory(d,index_type,metric_type)    #等价于 faiss.IndexFlatIP(d)     
print('index.is_trained=',index.is_trained)    # 输出为True，代表该类index不需要训练，只需要add向量进去即可
index.train(xb)
index.add(xb)                                      # 将向量库中的向量加入到index中
print('index.ntotal=',index.ntotal,'\n')           # 输出index中包含的向量总数，为100000 


# 四，相似向量查询
print('============================== 4, search ==============================')
k = 4                       # topK的K值
D, I = index.search(xq, k)  # xq为待检索向量，

# 返回的I为每个待检索query最相似TopK的索引list，
# D为其对应的距离/得分metric(distances/scores) to query

# I.shape (1000, 4) 1000个查询，每个查询返回四个 index
print('nearest vector ids:\n',I[:5],'\n')
# nearest vector ids:
#  [[ 207  381 1394 1019]
#  [ 300  911  142  526]
#  [ 838 1541  527  148]
#  [ 196  359  184  466]
#  [ 526  120  917  765]] 

# D.shape (1000, 4) 1000个查询，每个查询返回四个 scores
print('metric(distances/scores) to query:\n',D[-5:],'\n')
# metric(distances/scores) to query:
#  [[0.87687665 0.861282   0.8566772  0.85451   ]
#  [0.8702938  0.86668825 0.8593493  0.8523142 ]
#  [0.862915   0.8580744  0.8538497  0.84994495]
#  [0.86920005 0.86600465 0.8647547  0.86346215]
#  [0.8539624  0.8491495  0.84744585 0.8432568 ]] 


# 五，增删索引向量
print('============================== 5, add&remove ==============================')
xa = np.random.random((10000, d)).astype('float32')
xa[:, 0] += np.arange(len(xa)) / 1000.                
faiss.normalize_L2(xa)                            # 正则化
index.add(xa)                                     # 添加索引
print('after add, index.ntotal=',index.ntotal)    
# after add, index.ntotal= 110000

index.remove_ids(np.arange(1000,1111))            # 删除索引
print('after remove, index.ntotal=',index.ntotal,'\n') 
# after remove, index.ntotal= 109889 


# 六，保存加载索引
print('============================== 6, write&read ==============================')
faiss.write_index(index, "large.index")           # 保存索引
index_loaded = faiss.read_index('large.index')    # 加载保存的索引
print('index_loaded.ntotal=', index_loaded.ntotal)
# index_loaded.ntotal= 109889

# 结果保存为 large.index

在本次竞赛中，我们可以将 PDF_name 中关键字段作为 储存向量库；Question 作为查询向量，利用 Faiss 完成匹配工作。


##### 3.Faiss 应用

In [237]:
# 构建 储存向量库

# 获取
pdf_name_alllists = df_pdf_name['pdf_name'].apply(get_usableinfo_from_pdf_name).tolist()
pdf_name_alllists
# ['数源科技股份有限公司#数源科技#2020年',
#  '合兴汽车电子股份有限公司#合兴股份#2020年,
#  ...
# ]

['数源科技股份有限公司#数源科技#2020年',
 '合兴汽车电子股份有限公司#合兴股份#2020年',
 '昆吾九鼎投资控股股份有限公司#九鼎投资#2020年',
 '嘉凯城集团股份有限公司#嘉凯城#2019年',
 '北大医药股份有限公司#北大医药#2019年',
 '西藏珠峰资源股份有限公司#西藏珠峰#2021年',
 '山东惠发食品股份有限公司#惠发食品#2019年',
 '浙江东方基因生物制品股份有限公司#东方生物#2021年',
 '浙江田中精机股份有限公司#田中精机#2019年',
 '晶科电力科技股份有限公司#晶科科技#2020年',
 '张家港海锅新能源装备股份有限公司#海锅股份#2021年',
 '江苏蔚蓝锂芯股份有限公司#蔚蓝锂芯#2021年',
 '远大产业控股股份有限公司#远大控股#2021年',
 '深圳市有方科技股份有限公司#有方科技#2021年',
 '杭州远方光电信息股份有限公司#远方信息#2020年',
 '湖北盛天网络技术股份有限公司#盛天网络#2021年',
 '安徽省皖能股份有限公司#皖能电力#2019年',
 '好想你健康食品股份有限公司#好想你#2021年',
 '浙商中拓集团股份有限公司#浙商中拓#2020年',
 '多伦科技股份有限公司#多伦科技#2021年',
 '深圳市瑞凌实业集团股份有限公司#瑞凌股份#2019年',
 '浙江瀚叶股份有限公司#瀚叶股份#2020年',
 '新兴铸管股份有限公司#新兴铸管#2020年',
 '西宁特殊钢股份有限公司#西宁特钢#2021年',
 '北京东方园林环境股份有限公司#东方园林#2019年',
 '江西海源复合材料科技股份有限公司#海源复材#2020年',
 '崇达技术股份有限公司#崇达技术#2021年',
 '东阿阿胶股份有限公司#东阿阿胶#2019年',
 '安井食品集团股份有限公司#安井食品#2019年',
 '浙江司太立制药股份有限公司#司太立#2019年',
 '山东新华锦国际股份有限公司#新华锦#2019年',
 '中国船舶重工集团海洋防务与信息对抗股份有限公司#中国海防#2019年',
 '杭州迪普科技股份有限公司#迪普科技#2021年',
 '上海保隆汽车科技股份有限公司#保隆科技#2020年',
 '株洲旗滨集团股份有限公司#旗滨集团#2020年',
 

In [238]:
pdf_name_allembs = get_list_emb(pdf_name_alllists)

100%|█████████████████████████████████████████████████████████| 11588/11588 [05:48<00:00, 33.24it/s]


In [249]:
pdf_name_allembs[0].size, pdf_name_allembs[0]

(1024,
 array([-0.0272032 ,  0.01918118, -0.01478902, ...,  0.02082741,
         0.0675262 , -0.0028044 ], dtype=float32))

In [250]:
# 基本参数设置
d = 1024                                         # 向量维度  pdf_name_allembs[0].size
index_type = 'Flat'                              # index 类型
metric_type = faiss.METRIC_INNER_PRODUCT         # 度量(相似度/距离)类型

# 构建向量库索引
print('============================== 3,create&train ==============================')
index_pdf_names = faiss.index_factory(d,index_type,metric_type)    #等价于 faiss.IndexFlatIP(d)     
print('index.is_trained=',index_pdf_names.is_trained)        # 输出为True，代表该类index不需要训练，只需要add向量进去即可

index_pdf_names.train(pdf_name_allembs)
index_pdf_names.add(pdf_name_allembs)                        # 将向量库中的向量加入到index中
print('index.ntotal=',index_pdf_names.ntotal,'\n')           # 输出index中包含的向量总数，为100000 

# 保存加载索引
print('============================== 6, write&read ==============================')
faiss.write_index(index_pdf_names, "data/index_pdf-name.index")
print('index_loaded.ntotal =', index_pdf_names.ntotal)

============================== 3,create&train ==============================
index.is_trained= True
index.ntotal= 11588 

============================== 6, write&read ==============================
index_loaded.ntotal= 11588


In [244]:
# 问题 构建查询向量
q_lists_stopwords = [replace_stopwords(i, stopwords, special_) for i in tqdm(q_all_lists, ncols=100)]
q_lists_allembs = get_list_emb(q_lists_stopwords)
len(q_lists_allembs)

100%|███████████████████████████████████████████████████████████| 4700/4700 [02:23<00:00, 32.77it/s]


4700

In [246]:
q_lists_stopwords[:5]

['#2020年金宇生物技术股份有限公司#给我简要介绍一下报告期内公司#社会责任工作情况#',
 '#江化微2019年#简要介绍报告期内公司主要销售客户#客户集中度情况#并结合同行业情况进行分析#',
 '2019年四方科技电子信箱#是#',
 '康希诺生物股份公司#2020年#资产负债比率具体#是#至#',
 '苏州科达#2020年#博士及以上人员数量#是#']

In [247]:
q_lists_allembs[:5]

array([[-0.01341511, -0.00528573, -0.03538777, ...,  0.0025496 ,
        -0.00716865,  0.00021384],
       [-0.00991932,  0.00800164, -0.05759814, ..., -0.00765916,
         0.00493081, -0.00085113],
       [-0.00174594, -0.02006687, -0.04242305, ..., -0.01172827,
         0.03836973, -0.02767033],
       [-0.01030335,  0.02388459, -0.0207535 , ..., -0.00407091,
         0.04069532,  0.01142531],
       [-0.03861972, -0.02695625, -0.0635932 , ..., -0.03689505,
         0.08515219, -0.05573268]], dtype=float32)

In [291]:
# 相似向量查询
print('============================== 4, search ==============================')
k = 100                                             # topK的K值
D, I = index_pdf_names.search(q_lists_allembs, k)   # xq为待检索向量，返回的I为每个待检索query最相似TopK的索引list，D为其对应的距离

============================== 4, search ==============================


In [285]:
print('nearest vector ids:\n',I[:5],'\n')

nearest vector ids:
 [[10785  7782  1258  8991   640   147   189  9249  9597  2172]
 [ 1677  5331  2573    94 10976  5906 10384  1825  4791  2961]
 [10692   291  7551  1382  4800   256  1352  2800  9759 10668]
 [ 2679  8037  4497 10836  4365 11352  2255  1078 10769 11003]
 [ 2015  1813  1815  8669  7960  7662  8596  7764  8376  5503]] 



In [286]:
print('nearest vector ids:\n',D[:5],'\n')

nearest vector ids:
 [[0.75949585 0.73155093 0.71988326 0.71846646 0.7146287  0.7098887
  0.70535874 0.6903855  0.6885517  0.6858068 ]
 [0.6905894  0.6373172  0.6296762  0.62890035 0.61328626 0.60742915
  0.6045306  0.59699005 0.59662783 0.5950674 ]
 [0.7668812  0.68065685 0.6725097  0.66561246 0.6621757  0.6502046
  0.64706117 0.6336173  0.6072404  0.60483986]
 [0.8764615  0.8166691  0.6941056  0.69150364 0.68297786 0.68170357
  0.6709806  0.67004    0.66324556 0.6517556 ]
 [0.6824061  0.67169964 0.6443689  0.63767993 0.6284869  0.6158288
  0.61477816 0.6146865  0.61316013 0.61306006]] 



In [328]:
# 查看结果
# 前五个问题 余弦相似度最高的 pdf索引
I[:5,:1].T.tolist()[0]

[10785, 1677, 10692, 2679, 2015]

In [333]:
for iq, (idx, ipdf) in zip(q_lists_stopwords[:9], df_pdf_name.loc[I[:9,:1].T.tolist()[0]].iterrows()):
    print('%s\t\t%s'%(iq, ipdf[["all_name","short_name"]].tolist()))

#2020年金宇生物技术股份有限公司#给我简要介绍一下报告期内公司#社会责任工作情况#		['金宇生物技术股份有限公司', '生物股份']
#江化微2019年#简要介绍报告期内公司主要销售客户#客户集中度情况#并结合同行业情况进行分析#		['江阴江化微电子材料股份有限公司', '江化微']
2019年四方科技电子信箱#是#		['四方科技集团股份有限公司', '四方科技']
康希诺生物股份公司#2020年#资产负债比率具体#是#至#		['康希诺生物股份公司', '康希诺']
苏州科达#2020年#博士及以上人员数量#是#		['江苏博俊工业科技股份有限公司', '博俊科技']
河北金牛化工股份有限公司2020年法定代表人对比2019年#是否相同#		['河北建新化工股份有限公司', '建新股份']
南京康尼机电股份有限公司2019年企业研发经费与利润比值#是#2#		['南京康尼机电股份有限公司', '康尼机电']
2021年商业城现金及现金等价物增长率#是#2#		['永辉超市股份有限公司', '永辉超市']
平潭发展#2021年#投资收益增长率#到#是#		['中福海峡平潭发展股份有限公司', '平潭发展']


从上面结果我们可以看到：
- 苏州科达#2020年#博士及以上人员数量#是#     ['江苏博俊工业科技股份有限公司', '博俊科技']
- 2021年商业城现金及现金等价物增长率#是#2#    ['永辉超市股份有限公司', '永辉超市']

是有错误的，接下来考虑如何解决这个问题。



##### 4.Faiss 优化

在 判断 str_[short_name] 中每一个字符 是都在问题question中

In [332]:
def is_str_in_question_len1(q:str, str_:str):
    """
    判断 str_[short_name] 中每一个字符 是都在问题q中

    """
    cnt = 0
    for i in list(str_):
        if i in q:
            cnt += 1
    return cnt, cnt/len(str_)

def get_pdf_name_by_shortname(q:str, df_pdf:pd.DataFrame, idxs:list):
    """
    根据 df_pdf 的short_name 找到最相似的pdf索引
    """
    cnts = []
    for i in idxs:
        cnt, ratio = is_str_in_question_len1(q, df_pdf.loc[i, 'short_name'])
        # print(q, df_pdf.loc[i, 'short_name'], cnt, ratio)
        if ratio == 1:
            return i
        cnts.append(cnt)
    return cnts.index(max(cnts))

    
for idx, iq in enumerate(q_lists_stopwords[:9]):
    qidx = get_pdf_name_by_shortname(iq, df_pdf_name, I[idx,])
    print(idx, iq, df_pdf_name.loc[qidx, ["all_name", 'short_name', 'year']].to_list())



0 #2020年金宇生物技术股份有限公司#给我简要介绍一下报告期内公司#社会责任工作情况# ['金宇生物技术股份有限公司', '生物股份', '2020年']
1 #江化微2019年#简要介绍报告期内公司主要销售客户#客户集中度情况#并结合同行业情况进行分析# ['江阴江化微电子材料股份有限公司', '江化微', '2019年']
2 2019年四方科技电子信箱#是# ['四方科技集团股份有限公司', '四方科技', '2019年']
3 康希诺生物股份公司#2020年#资产负债比率具体#是#至# ['康希诺生物股份公司', '康希诺', '2020年']
4 苏州科达#2020年#博士及以上人员数量#是# ['苏州科达科技股份有限公司', '苏州科达', '2020年']
5 河北金牛化工股份有限公司2020年法定代表人对比2019年#是否相同# ['北方化学工业股份有限公司', '北化股份', '2020年']
6 南京康尼机电股份有限公司2019年企业研发经费与利润比值#是#2# ['南京康尼机电股份有限公司', '康尼机电', '2019年']
7 2021年商业城现金及现金等价物增长率#是#2# ['沈阳商业城股份有限公司', '商业城', '2021年']
8 平潭发展#2021年#投资收益增长率#到#是# ['中福海峡平潭发展股份有限公司', '平潭发展', '2021年']


之前的 匹配错误如下：
- 苏州科达#2020年#博士及以上人员数量#是#     ['江苏博俊工业科技股份有限公司', '博俊科技']
- 2021年商业城现金及现金等价物增长率#是#2#    ['永辉超市股份有限公司', '永辉超市']
1.受查询指标（博士）干扰
现在已经得到改正：
- 4 苏州科达#2020年#博士及以上人员数量#是# ['苏州科达科技股份有限公司', '苏州科达', '2020年']
- 7 2021年商业城现金及现金等价物增长率#是#2# ['沈阳商业城股份有限公司', '商业城', '2021年']
1.用短字符匹配重排

接下来我们将匹配结果储存起来，合并到 df_tests_pdf 中.


In [337]:
df_tests_pdf.head(10)

,question,year_list,year_count,q_type
q_idx,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf
2,2019年四方科技电子信箱是什么?,2019,1,pdf
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。,2019,1,pdf
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。,2021,1,pdf
9,平潭发展在2021年的投资收益增长率保留到小数点后两位是多少？,2021,1,pdf


In [339]:
df_tests_pdf['year_count'].value_counts()

year_count
1    4633
2      67
Name: count, dtype: int64

观察上面的 数据我们可以发现，有的 year_count 是由两年的，但是我们只找到了一年的 pdf 文件，该如何进行存储呢？
先观察一下两年的 questio 特点。

In [340]:
df_tests_pdf[df_tests_pdf['year_count']==2]

,question,year_list,year_count,q_type
q_idx,,,,
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf
214,请对比2020年和2021年，永清环保股份有限公司的法定代表人是否相同?,"2020,2021",2,pdf
289,对比2020年，2021年中航电子的法定代表人是否相同?,"2020,2021",2,pdf
368,在2020年，上海开开实业股份有限公司的法定代表人与2019年相比相同吗?,"2019,2020",2,pdf
490,山东阳谷华泰化工股份有限公司在2019-2021年间，法定代表人是否都相同?,"2019,2021",2,pdf
...,...,...,...,...
4617,中金岭南2021年的法定代表人与2020年相比是否相同？,"2020,2021",2,pdf
4672,上海华鑫股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf
4738,对比2020年，2021年江苏蔚蓝锂芯股份有限公司的法定代表人是否相同?,"2020,2021",2,pdf


In [ ]:
可以发现，year_count == 2 的数据，都是同一家公司下的，因此，我们知道了一家就拿到了两家。

In [372]:
df_pdf_name.head()

,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
0,2021-04-24__数源科技股份有限公司__000909__数源科技__2020年__年...,2021-04-24,数源科技股份有限公司,909,数源科技,2020年,年度报告
1,2021-04-19__合兴汽车电子股份有限公司__605005__合兴股份__2020年_...,2021-04-19,合兴汽车电子股份有限公司,605005,合兴股份,2020年,年度报告
2,2021-04-15__昆吾九鼎投资控股股份有限公司__600053__九鼎投资__2020...,2021-04-15,昆吾九鼎投资控股股份有限公司,600053,九鼎投资,2020年,年度报告
3,2020-04-30__嘉凯城集团股份有限公司__000918__嘉凯城__2019年__年...,2020-04-30,嘉凯城集团股份有限公司,918,嘉凯城,2019年,年度报告
4,2020-05-29__北大医药股份有限公司__000788__北大医药__2019年__年...,2020-05-29,北大医药股份有限公司,788,北大医药,2019年,年度报告


In [403]:
for ii, (idx, iq) in enumerate(tqdm(df_tests_pdf.iterrows(), ncols=100)):
    qidx = get_pdf_name_by_shortname(q_lists_stopwords[ii], df_pdf_name, I[ii,])
    # # 拿到 问题 对应的 pdf idx
    all_name = df_pdf_name.loc[qidx, 'all_name']
    # # 拿到 问题 对应的 公司名称（没有年份等其他信息）
    years = df_tests_pdf.loc[idx, 'year_list'] # <class 'str'>
    # # 拿到 问题 对应的 年份信息

    # 用 年份 与 名称 获得 问题 对应的
    years = [int(years)] if ',' not in years else [int(year) for year in years.split(',')]

    pdf_lists = [str(idxpdf) for idxpdf, itempdf in df_pdf_name[df_pdf_name['all_name'] == all_name].iterrows() \
                            if int(itempdf['year'][:-1]) in years]
    # print(idx, q_lists_stopwords[ii], '\n\t', iq['question'], df_tests_pdf.loc[idx, 'year_list'])
    # print(all_name)
    # print(pdf_lists)
    df_tests_pdf.loc[idx, 'pdf_idx'] = ','.join(pdf_lists)


4700it [00:09, 478.40it/s]


In [404]:
# 6 河北金牛化工股份有限公司2020年法定代表人对比2019年#是否相同# 
# 	 河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同? 2019,2020
# 北方化学工业股份有限公司
# ['8461', '9855']

df_pdf_name.loc[[8461, 9855]]

,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
8461,2020-04-28__北方化学工业股份有限公司__002246__北化股份__2019年_...,2020-04-28,北方化学工业股份有限公司,2246,北化股份,2019年,年度报告
9855,2021-04-23__北方化学工业股份有限公司__002246__北化股份__2020年_...,2021-04-23,北方化学工业股份有限公司,2246,北化股份,2020年,年度报告


In [405]:
df_tests_pdf.head(10)

,question,year_list,year_count,q_type,pdf_idx
q_idx,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,10785
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,1677
2,2019年四方科技电子信箱是什么?,2019,1,pdf,10692
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,2679
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,1813
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf,"8461,9855"
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。,2019,1,pdf,2853
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。,2021,1,pdf,8441
9,平潭发展在2021年的投资收益增长率保留到小数点后两位是多少？,2021,1,pdf,5895


至此，我们将 处理完成所有的概念问题以及PDF_name 与 Question 对应问题

In [7]:
# fn = "output/0.df_tests_pdf-%s.csv"%stamp()   # 对每份文件加入时间戳，防止覆盖
# df_tests_pdf.to_csv(fn, sep=";", index_label="q_idx")
# print(fn)
# output/0.df_tests_pdf-20230907-181225.csv

# df_tests = pd.read_csv("output/0.df_tests_pdf-20230907-181225.csv", sep=";", index_col="q_idx")
# df_tests.head()

,question,year_list,year_count,q_type,pdf_idx
q_idx,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,10785
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,1677
2,2019年四方科技电子信箱是什么?,2019,1,pdf,10692
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,2679
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,1813


#### 五、对应关系正确性检验

##### 1.Embedding 结果检验

在上述操作过程中，我们只能通过 Embedding 余弦相似度来进行 结果的筛选。<br>
这里就存在一个问题，难道 余弦相似度 最高的一定正确吗？<br>

这里我们采用 简称short_name 是否在 Question 来检验一下：

In [8]:
df_pdf_name.head()

,pdf_name,pub_time,all_name,code,short_name,year,type
pdf_idx,,,,,,,
0,2021-04-24__数源科技股份有限公司__000909__数源科技__2020年__年...,2021-04-24,数源科技股份有限公司,909,数源科技,2020年,年度报告
1,2021-04-19__合兴汽车电子股份有限公司__605005__合兴股份__2020年_...,2021-04-19,合兴汽车电子股份有限公司,605005,合兴股份,2020年,年度报告
2,2021-04-15__昆吾九鼎投资控股股份有限公司__600053__九鼎投资__2020...,2021-04-15,昆吾九鼎投资控股股份有限公司,600053,九鼎投资,2020年,年度报告
3,2020-04-30__嘉凯城集团股份有限公司__000918__嘉凯城__2019年__年...,2020-04-30,嘉凯城集团股份有限公司,918,嘉凯城,2019年,年度报告
4,2020-05-29__北大医药股份有限公司__000788__北大医药__2019年__年...,2020-05-29,北大医药股份有限公司,788,北大医药,2019年,年度报告


In [47]:
df_tests.head(5)

,question,year_list,year_count,q_type,pdf_idx
q_idx,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,10785
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,1677
2,2019年四方科技电子信箱是什么?,2019,1,pdf,10692
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,2679
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,1813


In [46]:
na_list, notin_list, emb_find = [], [], []
# na_list    空值 Embedding 没有匹配到合适的结果
# notin_list Embedding 匹配到结果但不包含shortname
# emb_find   包含shortname且Embedding
for idx, item in df_tests.iterrows():
    if isinstance(item['pdf_idx'], str):
        pdf_idx = int(item['pdf_idx']) if ',' not in item['pdf_idx'] else int(item['pdf_idx'].split(',')[0])
        # item['pdf_idx'] 可能性存在两个 数值 ex. 8461,9855
        short_name = df_pdf_name.loc[pdf_idx, 'short_name']
        if short_name not in item['question']:
            notin_list.append(idx)
        else:
            emb_find.append(idx)
    else:
        na_list.append(idx)
print("空值 Embedding 没有匹配到合适的结果:",len(na_list))
print("Embedding 匹配到结果但不包含shortname:", len(notin_list))
print("包含shortname且Embedding:",len(emb_find))


空值 Embedding 没有匹配到合适的结果: 315
Embedding 匹配到结果但不包含shortname: 953
包含shortname且Embedding: 3432


##### 2.尝试直接使用 字符包含直接匹配

In [158]:
all_allname_list = df_pdf_name['all_name'].tolist()
print(all_allname_list[:3])

all_shortname_lists = df_pdf_name['short_name'].tolist()
print(all_shortname_lists[:3])

['数源科技', '合兴股份', '九鼎投资']
['数源科技股份有限公司', '合兴汽车电子股份有限公司', '昆吾九鼎投资控股股份有限公司']


In [159]:
def find_name_in_Q(df_tests, name_lists):
    refind_list, reno_list = [], []

    for idx, item in tqdm(df_tests.iterrows(), ncols=100):
        ifind = ''
        for i in name_lists:
            if i in item['question']:
                ifind = i
                break
        if ifind != '':
            refind_list.append((idx, ifind))
        else:
            reno_list.append(idx)
    return refind_list, reno_list

In [160]:
allname_refind_list, allname_reno_list = find_name_in_Q(df_tests, all_allname_list)
print(len(allname_refind_list), len(allname_reno_list))

4700it [01:17, 60.28it/s]

2289 2411


In [161]:
shortname_refind_list, shortname_reno_list = find_name_in_Q(df_tests, all_shortname_lists)
print(len(shortname_refind_list), len(shortname_reno_list))

4700it [00:46, 101.85it/s]

4051 649


Embedding 结果说明：
- 包含shortname且Embedding: 3432 emb_find<br>
(认为是正确的)
- 有 953 个 简称是不在问题中的 notin_list <br>
(可能通过全称匹配)
-  存在 315 空值，也就是没有匹配到结果 na_list <br>
(可能不存在对应关系，即没有该问题中涉及到的pdf)

全称字符包含直接匹配  结果说明：
-  有 2289 个 通过 字符包含直接匹配匹配成功 allname_refind_list<br>
(认为是正确的)
-  有 2411 个是 无法通过包含匹配的 allname_reno_list


简称字符包含直接匹配  结果说明：
-  有 4051 个 通过 字符包含直接匹配匹配成功 shortname_refind_list
-  有 649 个是 无法通过包含匹配的 shortname_reno_list

接下来，我们对空值进行处理：

In [163]:
# 三种方法同时判定为空值的idx
na3_lists = [*(set(na_list) & set(allname_reno_list) & set(shortname_reno_list))]   # ===> 37
print("三种方法同时判定为空值的个数 ===> ", len(na3_lists))

三种方法同时判定为空值的个数 ===>  0


##### 3.互补方法确定结果

得到了上述的两种结果，我们可以得出：有 3432 个是正确的，而37个空值，<br>
其他的如何确定呢？
我们现在有两种匹配格式，正则（简称包含）和 Embedding 近似匹配。<br>
结合两者特点：正则匹配通常是精确匹配, Embedding近似匹配可以容忍语义级别的误差。<br><br>

因此，我们可以按照这个优先级别：
1. all_name 全称包含的一定对
2. 包含shortname且Embedding emb_find
3. shortname包含直接匹配匹配成功 refind_list
4. Embedding 匹配到结果但不包含shortname notin_list
5. 两种方法同时判定为空值的个数 reno_list


In [111]:
df_tests.head(10)

,question,year_list,year_count,q_type,pdf_idx
q_idx,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,10785
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,1677
2,2019年四方科技电子信箱是什么?,2019,1,pdf,10692
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,2679
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,1813
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf,"8461,9855"
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。,2019,1,pdf,2853
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。,2021,1,pdf,8441
9,平潭发展在2021年的投资收益增长率保留到小数点后两位是多少？,2021,1,pdf,5895


In [178]:
df_tests_correct = df_tests.copy()
# 删除存在的 pdf_idx， 
# 并添加 short_name 记录每一步, 因为 可能存在 2个pdf_idx，所以保留 shortname 在获得 pdf_idx
#   避免 pdf_idx 修改前后数量上不一致
df_tests_correct['pdf_idx'], df_tests_correct['short_name'] = '', np.nan
df_tests_correct.head()

,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,,NaN
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,,NaN
2,2019年四方科技电子信箱是什么?,2019,1,pdf,,NaN
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,,NaN
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,,NaN


In [174]:
def correct_idx(idx_list:list, done_list:list, df_tests:pd.DataFrame, 
                df_tests_correct:pd.DataFrame) -> pd.DataFrame:
    replace_idxs = []
    for i in idx_list:           # 本次需要替换的df_test.q_idx
        if i not in done_list:   # 对已经完成的不再进行替换，保证优先级
            replace_idxs.append(i)
            temp = df_tests.loc[i, 'pdf_idx']
            pdf_idx = int(temp) if ',' not in temp else int(temp.split(',')[0]) 
            df_tests_correct.loc[i, 'short_name'] = df_pdf_name.loc[pdf_idx, 'short_name']
    return df_tests_correct, replace_idxs

def get_done_index(df:pd.DataFrame, col:str):
    return df[df[col].apply(lambda x: isinstance(x, str))].index.tolist()

In [179]:
# 1. all_name 全称包含的一定对
# all_allname_list[:3]
replace_idxs = []
for i, j in allname_refind_list:
    short_name = df_pdf_name[df_pdf_name['all_name'] == j]['short_name'].to_list()[0]
    replace_idxs.append(i)
    df_tests_correct.loc[i, 'short_name'] = short_name
print(len(replace_idxs))
df_tests_correct.loc[replace_idxs]

2289


,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,,生物股份
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,,康希诺
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf,,金牛化工
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。,2019,1,pdf,,康尼机电
11,安记食品股份有限公司在2019年的营业利润率是多少，结果请保留至小数点后两位？,2019,1,pdf,,安记食品
...,...,...,...,...,...,...
4990,2020年苏州天准科技股份有限公司企业硕士及以上人员占职工人数比例为多少?保留2位小数。,2020,1,pdf,,天准科技
4994,2021年鹏欣环球资源股份有限公司总资产增长率为多少?保留2位小数。,2021,1,pdf,,鹏欣资源
4995,苏州东微半导体股份有限公司在2021年归属于母公司所有者的净利润是多少元?,2021,1,pdf,,东微半导


In [180]:
done_lists = get_done_index(df_tests_correct, col='short_name')
print(len(done_lists))

2289


In [181]:
# 2. 包含shortname且Embedding emb_find
df_tests_correct, replace_idxs =correct_idx(emb_find, done_lists, df_tests, df_tests_correct)
print(len(replace_idxs))
df_tests_correct.loc[replace_idxs]

1961


,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,,江化微
2,2019年四方科技电子信箱是什么?,2019,1,pdf,,四方科技
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,,苏州科达
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。,2021,1,pdf,,商业城
9,平潭发展在2021年的投资收益增长率保留到小数点后两位是多少？,2021,1,pdf,,平潭发展
...,...,...,...,...,...,...
4988,龙软科技2020年对联营企业和合营企业的投资收益是多少元?,2020,1,pdf,,龙软科技
4989,派生科技在2021年的应收款项融资有多少元?,2021,1,pdf,,派生科技
4991,2021年东方海洋的营业收入增长率是多少？请保留两位小数。,2021,1,pdf,,东方海洋


In [182]:
done_lists = get_done_index(df_tests_correct, col='short_name')
print(len(done_lists))

4250


In [183]:
# 3. 字符包含直接匹配匹配成功 refind_list
replace_idxs = []
for (i,j) in shortname_refind_list:
    if i not in done_lists:
        replace_idxs.append(i)
        df_tests_correct.loc[i, 'short_name'] = j
print(len(replace_idxs))
df_tests_correct.loc[replace_idxs[:5]]

,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,
10,请问，光云科技2019年的财务费用是多少元?,2019,1,pdf,,光云科技
13,2019年爱丽家居企业名称是什么?,2019,1,pdf,,爱丽家居
29,维康药业2019年企业名称是什么?,2019,1,pdf,,维康药业
59,精准信息2021年的其他非流动金融资产是多少元？,2021,1,pdf,,精准信息
61,中国太保2019年的资产负债比率是多少?保留2位小数。,2019,1,pdf,,中国太保


In [184]:
done_lists = get_done_index(df_tests_correct, col='short_name')
print(len(done_lists))

4647


In [185]:
# 4. Embedding 匹配到结果但不包含shortname notin_list
df_tests_correct, replace_idxs =correct_idx(notin_list, done_lists, df_tests, df_tests_correct)
df_tests_correct.loc[replace_idxs[:5]]

,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,
183,福建龙溪轴承(集团)股份有限公司2021年的投资收益增长率是多少？请保留2位小数。,2021,1,pdf,,龙溪股份
188,乐鑫信息科技(上海)股份有限公司2019年营业外支出和营业外收入分别是多少元?,2019,1,pdf,,乐鑫科技
210,清研环境2021年每股经营现金流量是多少元?,2021,1,pdf,,好想你
215,2021年重庆长江造型材料(集团)股份有限公司年报数据中，负债合计是多少元?,2021,1,pdf,,长江材料
270,在2020年，甘肃国芳工贸(集团)股份有限公司的流动比率是多少？请保留至小数点后2位。,2020,1,pdf,,国芳集团


In [186]:
done_lists = get_done_index(df_tests_correct, col='short_name')
print(len(done_lists))

4700


In [187]:
# 4. 两种方法同时判定为空值的个数 reno_list

na_final_lists = df_tests_correct[df_tests_correct['short_name'].apply(lambda x: not isinstance(x, str))].index.to_list()
print(len(na_final_lists))
df_tests_correct.loc[na_final_lists]

0


,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,


至此，我们已经通过 字符串匹配 + Embedding 余弦相似度 匹配获得完整的匹配结果。<br>让我们看看结果：

In [188]:
df_tests_correct.head(10)

,question,year_list,year_count,q_type,pdf_idx,short_name
q_idx,,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,,生物股份
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,,江化微
2,2019年四方科技电子信箱是什么?,2019,1,pdf,,四方科技
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,,康希诺
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,,苏州科达
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf,,金牛化工
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。,2019,1,pdf,,康尼机电
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。,2021,1,pdf,,商业城
9,平潭发展在2021年的投资收益增长率保留到小数点后两位是多少？,2021,1,pdf,,平潭发展


In [204]:
df_tests_correct_up = df_tests_correct.copy()

In [205]:
# 补全 pdf_idx

for ii, (idx, iq) in enumerate(tqdm(df_tests_correct_up.iterrows(), ncols=100)):
    qidx = df_pdf_name[df_pdf_name['short_name'] == iq['short_name']].index.tolist()
    # print(qidx)
    # # 拿到 问题 对应的 pdf idx
    all_name = df_pdf_name.loc[qidx[0], 'all_name']
    # print(all_name)
    # # 拿到 问题 对应的 公司名称（没有年份等其他信息）
    years = df_tests_correct_up.loc[idx, 'year_list'] # <class 'str'>
    # print(years)
    # # 拿到 问题 对应的 年份信息

    # 用 年份 与 名称 获得 问题 对应的
    years = [int(years)] if ',' not in years else [int(year) for year in years.split(',')]

    pdf_lists = [int(idxpdf) for idxpdf, itempdf in df_pdf_name[df_pdf_name['all_name'] == all_name].iterrows() \
                            if int(itempdf['year'][:-1]) in years]
    pdfname_list = ','.join(df_pdf_name.loc[pdf_lists, 'pdf_name'].to_list()) if len(pdf_lists) != 0 else np.nan
    # print(all_name, years, pdf_lists, pdfname_list, iq['question'])
    # print(idx, q_lists_stopwords[ii], '\n\t', iq['question'], df_tests_pdf.loc[idx, 'year_list'])
    # print(all_name)
    # print(pdf_lists)
    df_tests_correct_up.loc[idx, 'pdf_idx'] = ','.join([str(i) for i in pdf_lists])
    df_tests_correct_up.loc[idx, 'pdf_name'] = pdfname_list


4700it [00:18, 260.78it/s]


In [207]:
df_tests_correct_up.head(10)

,question,year_list,year_count,q_type,pdf_idx,short_name,pdf_name
q_idx,,,,,,,
0,能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？,2020,1,pdf,10785,生物股份,2021-04-23__金宇生物技术股份有限公司__600201__生物股份__2020年_...
1,请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业...,2019,1,pdf,1677,江化微,2020-03-12__江阴江化微电子材料股份有限公司__603078__江化微__2019...
2,2019年四方科技电子信箱是什么?,2019,1,pdf,10692,四方科技,2020-04-16__四方科技集团股份有限公司__603339__四方科技__2019年_...
4,康希诺生物股份公司在2020年的资产负债比率具体是多少，需要保留至小数点后两位？,2020,1,pdf,2679,康希诺,2021-03-29__康希诺生物股份公司__688185__康希诺__2020年__年度报...
5,苏州科达在2020年的博士及以上人员数量是多少?,2020,1,pdf,1813,苏州科达,2021-03-23__苏州科达科技股份有限公司__603660__苏州科达__2020年_...
6,河北金牛化工股份有限公司2020年法定代表人对比2019年是否相同?,"2019,2020",2,pdf,"6637,7242",金牛化工,2020-03-05__河北金牛化工股份有限公司__600722__金牛化工__2019年_...
7,南京康尼机电股份有限公司2019年企业研发经费与利润比值是多少?保留2位小数。,2019,1,pdf,2853,康尼机电,2020-04-03__南京康尼机电股份有限公司__603111__康尼机电__2019年_...
8,2021年商业城现金及现金等价物增长率是多少?保留2位小数。,2021,1,pdf,8441,商业城,2022-03-12__沈阳商业城股份有限公司__600306__商业城__2021年__年...
9,平潭发展在2021年的投资收益增长率保留到小数点后两位是多少？,2021,1,pdf,5895,平潭发展,2022-04-27__中福海峡平潭发展股份有限公司__000592__平潭发展__2021...


In [214]:
nan_lists = df_tests_correct_up[df_tests_correct_up['pdf_name'].apply(lambda x: not isinstance(x, str))].index.to_list()
df_tests_correct_up.loc[nan_lists]

,question,year_list,year_count,q_type,pdf_idx,short_name,pdf_name
q_idx,,,,,,,
10,请问，光云科技2019年的财务费用是多少元?,2019,1,pdf,,光云科技,NaN
13,2019年爱丽家居企业名称是什么?,2019,1,pdf,,爱丽家居,NaN
17,2019年南亚新材料科技股份有限公司研发费用和财务费用分别是多少元?,2019,1,pdf,,南亚新材,NaN
25,2020年山东玉马遮阳科技股份有限公司净利润率为多少?保留2位小数。,2020,1,pdf,,玉马遮阳,NaN
29,维康药业2019年企业名称是什么?,2019,1,pdf,,维康药业,NaN
...,...,...,...,...,...,...,...
4946,金迪克2020年的营业外支出是多少元?,2020,1,pdf,,金迪克,NaN
4962,2019年明新旭腾的研发费用和财务费用分别为多少元？,2019,1,pdf,,明新旭腾,NaN
4975,2019年赛伍技术研发人员占职工人数比例是多少?保留2位小数。,2019,1,pdf,,赛伍技术,NaN


这些都是没有对应 pdf 的Question，在之后的过程中，这些问题可以直接回答不知道。<br>
值得注意的是，经SMP组委会确认：如果没有对应的 PDF 但是提交的结果却给出了明确的答案，会**扣分**

例如：请问，光云科技2019年的财务费用是多少元?
回答：不知道光云科技2019年的财务费用，因为没有给出相关文件。

In [215]:
# 导出结果
# fn = "output/0.pdf_name-%s.csv"%stamp()   # 对每份文件加入时间戳，防止覆盖
# df_tests_correct_up.to_csv(fn, sep=";", index_label="q_idx")
# print(fn)
# output/0.Question-20230901-162551.csv

# df_tests = pd.read_csv("output/0.Question-20230901-162551.csv", sep=";", index_col="q_idx")
# df_tests.head()

output/0.Question-20230901-162551.csv


- 补充实验：Embedding 准确率判断

In [228]:
# 在 df_tests 中 空值为 nan, df_tests_correct_up 中 空值 为 ''
emb_pdf_idx = [i if isinstance(i, str) else '' for i in df_tests['pdf_idx'].tolist()] # 控制同一
res_pdf_idx = df_tests_correct_up['pdf_idx'].tolist()
print(len(emb_pdf_idx), len(res_pdf_idx))
equal_lists = [i for i, j in zip(emb_pdf_idx, res_pdf_idx) if i == j]
print("Embedding 准确率为：%.2f%%, 正确个数 %d"%(len(equal_lists) / len(emb_pdf_idx) * 100, len(equal_lists)))


4700 4700
Embedding 准确率为：92.02%, 正确个数 4325


Embedding 准确率达到 92.02%，正确个数 4325。<br>
准确率能够达到 92.02%，说明 Embedding 模型已经能够很好的将问题与文件对应起来。<br>

在实际的项目过程中，出于 Embedding 本身的局限性，我们仍然需要人工 / 规则对结果进行判断，提升准确性，以更好的完善结果。